In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=5
%env DWAVE_API_TOKEN=4Fq8-98e991a786ff04f4f4b6ab5466629411358ce418
%config InlineBackend.figure_format = 'retina'
!nvidia-smi

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: DWAVE_API_TOKEN=4Fq8-98e991a786ff04f4f4b6ab5466629411358ce418
Thu Apr 17 18:45:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A6000               Off |   00000000:1B:00.0 Off |                  Off |
| 30%   43C    P2             75W /  300W |    9468MiB /  49140MiB |      0%      Default |
|                                      

In [2]:
import hydra
from hydra.utils import instantiate
from hydra import initialize, compose

import sys
import os
import getpass

os.chdir('/home/' + getpass.getuser() + '/CaloQVAE/')
sys.path.insert(1, '/home/' + getpass.getuser() + '/CaloQVAE/')

#external libraries
import os
import pickle
import datetime
import sys
import yaml
import json

import torch.optim as optim
import torch
from torch import nn
from torch.nn.functional import binary_cross_entropy_with_logits
torch.manual_seed(32)
import numpy as np
np.random.seed(32)
import matplotlib.pyplot as plt
import hydra
from hydra.utils import instantiate

from omegaconf import OmegaConf

# PyTorch imports
from torch import device, load, save
from torch.nn import DataParallel
from torch.cuda import is_available

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
    
# Weights and Biases
import wandb

#self defined imports
from CaloQVAE import logging

[18:45:36.357] INFO   CaloQVAE                                          Willkommen!
[18:45:36.358] INFO   CaloQVAE                                          Loading configuration.


In [3]:
logger = logging.getLogger(__name__)

from data.dataManager import DataManager
from utils.plotting.plotProvider import PlotProvider
from engine.engine import Engine
from models.modelCreator import ModelCreator

from utils.plotting.HighLevelFeatures import HighLevelFeatures as HLF
HLF_1_photons = HLF('photon', filename='/fast_scratch_1/caloqvae/data/atlas/binning_dataset_1_photons.xml', wandb=False)
HLF_1_pions = HLF('pion', filename='/fast_scratch_1/caloqvae/data/atlas/binning_dataset_1_pions.xml', wandb=False)
HLF_1_electron = HLF('electron', filename='/fast_scratch_1/caloqvae/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)

[18:45:37.932] INFO   numexpr.utils                                     Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[18:45:37.934] INFO   numexpr.utils                                     Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[18:45:37.935] INFO   numexpr.utils                                     NumExpr defaulting to 8 threads.
2025-04-17 18:45:38,648 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[18:45:38.648] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [4]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="configs")

hydra.initialize()

In [5]:
# config=compose(config_name="config.yaml")
config=compose(config_name="config.yaml")
wandb.init(project="caloqvae", entity="qvae", config=config, mode='disabled')
modelCreator = ModelCreator(cfg=config)
dataMgr = DataManager(cfg=config)
#initialise data loaders
dataMgr.init_dataLoaders()
#run pre processing: get/set input dimensions and mean of train dataset
dataMgr.pre_processing()

if config.model.activation_fct.lower()=="relu":
    modelCreator.default_activation_fct=torch.nn.ReLU()
elif config.model.activation_fct.lower()=="tanh":
    modelCreator.default_activation_fct=torch.nn.Tanh()
else:
    logger.warning("Setting identity as default activation fct")
    modelCreator.default_activation_fct=torch.nn.Identity()

#instantiate the chosen model
#loads from file 
model=modelCreator.init_model(dataMgr=dataMgr)

#create the NN infrastructure
model.create_networks()

/usr/local/lib/python3.10/dist-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
[18:45:41.490] INFO   data.dataManager                                  Loading Data
[18:46:03.366] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f998a34cb80>: 80000 events, 625 batches
[18:46:03.369] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9a48bd37c0>: 10000 events, 10 batches
[18:46:03.370] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9a474f9c60>: 10000 events, 10 batches
[18:46:05.305] INFO   models.modelCreator                               Initialising Model Type AtlasConditionalQVAE3DHD
2025-04-17 18:46:05,320 dw

Initializing NetworkV3
Layer Inputs:  [1208, 3368, 3368, 3368]
Layer Outputs:  [2160, 2160, 2160, 6480]
Raw Layer Indices:  [0, 2160, 2160, 2160, 6480]
Initializing NetworkV3
Initializing NetworkV3
Initializing NetworkV3
Initializing NetworkV3


In [6]:


# dev = torch.device("cuda:0")
dev = "cuda:{0}".format(config.gpu_list[0])
wandb.watch(model)
engine=instantiate(config.engine, config)
engine._config=config
#add dataMgr instance to engine namespace
engine.data_mgr=dataMgr
#add device instance to engine namespace
engine.device=dev    
#instantiate and register optimisation algorithm
engine.optimiser = torch.optim.Adam(model.parameters(),
                                    lr=config.engine.learning_rate)
#add the model instance to the engine namespace
engine.model = model
# add the modelCreator instance to engine namespace
engine.model_creator = modelCreator
engine.model = engine.model.to(dev)

[18:46:09.229] INFO   wandb                                             Watching
/home/luian1/.local/lib/python3.10/site-packages/coffea/util.py:154: FutureWarning: In coffea version v0.8.0 (target date: 31 Dec 2022), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)
[18:46:09.900] INFO   engine.engineAtlas                                Setting up engine Atlas.
[18:46:09.901] INFO   engine.engineCaloV3                               Setting up engine Calo.
[18:46:09.902] INFO   engine.engine                                     Setting up default engine.


In [7]:
train_loader,test_loader,val_loader = engine.data_mgr.create_dataLoader()

[18:46:30.875] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9a2d42a080>: 80000 events, 625 batches
[18:46:30.877] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f9a2d42a020>: 10000 events, 10 batches
[18:46:30.878] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f990c73b9a0>: 10000 events, 10 batches


In [8]:
def load_state(model, run_path, device):
        model_loc = run_path
        
        # Open a file in read-binary mode
        with open(model_loc, 'rb') as f:
            # Interpret the file using torch.load()
            checkpoint=torch.load(f, map_location=device)
            
            local_module_keys=list(model._modules.keys())
            print(local_module_keys)
            for module in checkpoint.keys():
                if module in local_module_keys:
                    print("Loading weights for module = ", module)
                    getattr(model, module).load_state_dict(checkpoint[module])

In [9]:
import sys
sys.path.append('/path/to/project')

import utils.plotting.HighLevelFeatures as HLF
import h5py
from ipynb.fs.full.utils.plotting.HEPMetrics import get_fpd_kpd_metrics

In [10]:
def search_HEPMetric(path_head, model_name, start_epoch, end_epoch, epoch_int):
    en_list = []
    fpd_recon, fpd_sample = [], []
    kpd_recon, kpd_sample = [], []
    hlf, ref_hlf = None, None
    for en in range(start_epoch, end_epoch, epoch_int):
        run_path = path_head + str(en) + ".pth"
        modelname = model_name
        datascaled = 'reduced'
        open_path = path_head.replace(path_head.split('/')[-1], '') + "config.yaml"
        with open(open_path, 'r') as file:
            model_config = yaml.safe_load(file)
            R = json.loads(model_config["_content"]["value"]['engine'].replace("'", "\""))['r_param']
            reducedata = True if model_config["_content"]["value"]["reducedata"] == 'True' else False
            scaled=False
            
        arch = config['model']['model_type']
        part = config['data']['particle_type']
        modelCreator.load_state(run_path, dev)
        engine.model.eval();

        # get the samples
        xtarget_samples = []
        xrecon_samples = []
        xgen_samples = []
        xgen_samples_qpu = []
        n_samples4_qpu = 200
        
        # xrecon_samples_2 = []
        
        # labelstarget_samples = []
        # labelsrecon_samples = []
        entarget_samples = []
        with torch.no_grad():
            for xx in train_loader:
            # for xx in train_loader:
                in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
                ###############################################
                # true_energy = true_energy[:n_samples4_qpu,:]
                # in_data = in_data[:n_samples4_qpu,:]
                ##############################################
                # print(in_data.shape)
                if reducedata:
                    in_data = engine._reduce(in_data, true_energy, R=R)
                fwd_output = engine.model((in_data, true_energy), False)
                if reducedata:
                    in_data = engine._reduceinv(in_data, true_energy, R=R)
                    recon_data = engine._reduceinv(fwd_output.output_activations, true_energy, R=R)
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy, num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
                    # sample_data_qpu = engine._reduceinv(sample_data_qpu, sample_energies_qpu, R=R)
                elif scaled:
                    in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
                    recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_just_act.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_energy_shift.detach().cpu().numpy()))
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy[:100,:], num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
                    # sample_data_qpu = torch.tensor(engine._data_mgr.inv_transform(sample_data_qpu.detach().cpu().numpy()))
                else:
                    in_data = in_data.detach().cpu()*1000
                    recon_data = fwd_output.output_activations.detach().cpu()*1000
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    # sample_energies, sample_data = engine._model.generate_samples(num_samples=2048)
                    sample_data = sample_data.detach().cpu()*1000
        
        
                xtarget_samples.append(in_data.detach().cpu())
                xrecon_samples.append( recon_data.detach().cpu())
                xgen_samples.append( sample_data.detach().cpu())
                # xgen_samples_qpu.append( sample_data_qpu.detach().cpu())
                entarget_samples.append(true_energy.detach().cpu())
        
                # xrecon_samples_2.append( recon_data_2.detach().cpu())
            for xx in val_loader:
            # for xx in train_loader:
                in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
                ###############################################
                # true_energy = true_energy[:n_samples4_qpu,:]
                # in_data = in_data[:n_samples4_qpu,:]
                ##############################################
                # print(in_data.shape)
                if reducedata:
                    in_data = engine._reduce(in_data, true_energy, R=R)
                fwd_output = engine.model((in_data, true_energy), False)
                if reducedata:
                    in_data = engine._reduceinv(in_data, true_energy, R=R)
                    recon_data = engine._reduceinv(fwd_output.output_activations, true_energy, R=R)
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy, num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
                    # sample_data_qpu = engine._reduceinv(sample_data_qpu, sample_energies_qpu, R=R)
                elif scaled:
                    in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
                    recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_just_act.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_energy_shift.detach().cpu().numpy()))
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy[:100,:], num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
                    # sample_data_qpu = torch.tensor(engine._data_mgr.inv_transform(sample_data_qpu.detach().cpu().numpy()))
                else:
                    in_data = in_data.detach().cpu()*1000
                    recon_data = fwd_output.output_activations.detach().cpu()*1000
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    # sample_energies, sample_data = engine._model.generate_samples(num_samples=2048)
                    sample_data = sample_data.detach().cpu()*1000
        
        
                xtarget_samples.append(in_data.detach().cpu())
                xrecon_samples.append( recon_data.detach().cpu())
                xgen_samples.append( sample_data.detach().cpu())
                # xgen_samples_qpu.append( sample_data_qpu.detach().cpu())
                entarget_samples.append(true_energy.detach().cpu())
        
                # xrecon_samples_2.append( recon_data_2.detach().cpu())
            for xx in test_loader:
            # for xx in train_loader:
                in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
                ###############################################
                # true_energy = true_energy[:n_samples4_qpu,:]
                # in_data = in_data[:n_samples4_qpu,:]
                ##############################################
                # print(in_data.shape)
                if reducedata:
                    in_data = engine._reduce(in_data, true_energy, R=R)
                fwd_output = engine.model((in_data, true_energy), False)
                if reducedata:
                    in_data = engine._reduceinv(in_data, true_energy, R=R)
                    recon_data = engine._reduceinv(fwd_output.output_activations, true_energy, R=R)
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy, num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
                    # sample_data_qpu = engine._reduceinv(sample_data_qpu, sample_energies_qpu, R=R)
                elif scaled:
                    in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
                    recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_just_act.detach().cpu().numpy()))
                    # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_energy_shift.detach().cpu().numpy()))
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    
                    if True:
                        sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy[:100,:], num_samples=1, thrsh=30, beta=1/beta0)
                    else:
                        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                        # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
                    # sample_data_qpu = torch.tensor(engine._data_mgr.inv_transform(sample_data_qpu.detach().cpu().numpy()))
                else:
                    in_data = in_data.detach().cpu()*1000
                    recon_data = fwd_output.output_activations.detach().cpu()*1000
                    engine._model.sampler._batch_size = true_energy.shape[0]
                    sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
                    engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
                    # sample_energies, sample_data = engine._model.generate_samples(num_samples=2048)
                    sample_data = sample_data.detach().cpu()*1000
        
        
                xtarget_samples.append(in_data.detach().cpu())
                xrecon_samples.append( recon_data.detach().cpu())
                xgen_samples.append( sample_data.detach().cpu())
                # xgen_samples_qpu.append( sample_data_qpu.detach().cpu())
                entarget_samples.append(true_energy.detach().cpu())
        
                # xrecon_samples_2.append( recon_data_2.detach().cpu())
            
            
        xtarget_samples = torch.cat(xtarget_samples, dim=0)
        xrecon_samples = torch.cat(xrecon_samples, dim=0)
        xgen_samples = torch.cat(xgen_samples, dim=0)
        # xgen_samples_qpu = torch.cat(xgen_samples_qpu, dim=0)
        entarget_samples = torch.cat(entarget_samples, dim=0)
        
        # xrecon_samples_2 = torch.cat(xrecon_samples_2, dim=0)
        if en == start_epoch:
            print("First epoch")
            hlf = HLF.HighLevelFeatures('electron', filename='/fast_scratch_1/caloqvae/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)
            ref_hlf = HLF.HighLevelFeatures('electron', filename='/fast_scratch_1/caloqvae/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)
            hlf.Einc = entarget_samples

        recon_HEPMetrics = get_fpd_kpd_metrics(np.array(xtarget_samples), np.array(xrecon_samples), False, hlf, ref_hlf)
        sample_HEPMetrics = get_fpd_kpd_metrics(np.array(xtarget_samples), np.array(xgen_samples), False, hlf, ref_hlf)

        en_list.append(en)
        fpd_recon.append(recon_HEPMetrics[0])
        kpd_recon.append(recon_HEPMetrics[2])
        fpd_sample.append(sample_HEPMetrics[0])
        kpd_sample.append(sample_HEPMetrics[2])
        print("Finished generating HEP Metrics for epoch " + str(en) + " ...")
    return en_list, fpd_recon, kpd_recon, fpd_sample, kpd_sample

In [11]:
# path_head = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240716_092439-ja6l8prq/files/AtlasConditionalQVAE3D_atlas_default_"
# # path_head = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240722_235537-ws2su3jq/files/AtlasConditionalQVAE3DHD_atlas_default_"
# path_head = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240717_230102-6jq8dbkm/files/AtlasConditionalQVAE3D_atlas_default_"
# path_head = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240809_192828-bk3n30hp/files/AtlasConditionalQVAE3DHD_atlas_default_"
# model_name = 'robust-serenity-216'
# path_head = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240717_230102-6jq8dbkm/files/AtlasConditionalQVAE3D_atlas_default_"
# model_name = 'mild-pond-223'
# HEPMetric_output = search_HEPMetric(path_head, model_name, 10, 151, 10)

In [12]:
# import matplotlib.pyplot as plt

# en_list, fpd_recon, kpd_recon, fpd_sample, kpd_sample = HEPMetric_output[0], HEPMetric_output[1], HEPMetric_output[2], HEPMetric_output[3], HEPMetric_output[4]
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# # Plot fpd_recon and fpd_sample on ax1
# ax1.scatter(en_list, fpd_recon, color='blue', label='FPD Recon')
# ax1.scatter(en_list, fpd_sample, color='green', label='FPD Sample')
# ax1.set_xlabel('Epoch Number')
# ax1.set_ylabel('FPD Values')
# ax1.set_title('FPD Recon vs FPD Sample')
# ax1.legend()

# # Plot kpd_recon and kpd_sample on ax2
# ax2.scatter(en_list, kpd_recon, color='blue', label='KPD Recon')
# ax2.scatter(en_list, kpd_sample, color='green', label='KPD Sample')
# ax2.set_xlabel('Epoch Number')
# ax2.set_ylabel('KPD Values')
# ax2.set_title('KPD Recon vs KPD Sample')
# ax2.legend()

# plt.tight_layout()
# plt.show()

In [13]:
# winter-snowflake-24: GumBoltAtlasPRBMCNN
# run_path = "/fast_scratch/sgonzalez/wandb/run-20240228_120832-q78nzust/files/GumBoltAtlasPRBMCNN_atlas_default_best.pth"

# run_path = "/fast_scratch/sgonzalez/wandb/run-20240313_155921-4q9i8pnt/files/GumBoltAtlasPRBMCNN_atlas_default_best.pth"
# modelname = 'macabre-candle-1372'
# datascaled = 'scaled'
# with open("/fast_scratch/sgonzalez/wandb/run-20240313_155921-4q9i8pnt/files/config.yaml", 'r') as file:
#     model_config = yaml.safe_load(file)
#     #R = json.loads(model_config["_content"]["value"]['engine'].replace("'", "\""))['r_param']
#     #reducedata = True if model_config["_content"]["value"]["reducedata"] == 'True' else False

# change this directory to load the synthetic data

# run_path = "/fast_scratch/QVAE/syn_data/dataset2_synthetic_macabre-candle-1372v2.hdf5"

# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/AtlasConditionalQVAEv2_atlas_default_best.pth"
# modelname = 'dutiful-silence-108'
# datascaled = 'scaled'
# with open("/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/config.yaml", 'r') as file:
#     model_config = yaml.safe_load(file)
#     #R = json.loads(model_config["_content"]["value"]['engine'].replace("'", "\""))['r_param']
#     #reducedata = True if model_config["_content"]["value"]["reducedata"] == 'True' else False


# arch = config['model']['model_type']
# part = config['data']['particle_type']
# print(arch)
# print(part)


# # load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))
# load_state(model, run_path, dev)
# model.eval();

#pretty-shape-455 | CNN + cond + scaled data + Cyl EncDec + lin/sqrt/log LONG energy encoded + CRBM 1st Partition Binv2 +scaled
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/AtlasConditionalQVAEv2_atlas_default_best.pth"
# # run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# modelname = 'dutiful-silence-108'
# datascaled = 'reduced'
# with open("/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/config.yaml", 'r') as file:
#     model_config = yaml.safe_load(file)
#     R = json.loads(model_config["_content"]["value"]['engine'].replace("'", "\""))['r_param']
#     reducedata = True if model_config["_content"]["value"]["reducedata"] == 'True' else False
#     scaled=False

# 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240628_222152-jbtlda15/files/AtlasConditionalQVAEv2_atlas_default_best.pth"
# Dutiful-Silence-108 4 subdecoders on 0.001
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240620_223442-7ggy3s19/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# Rare-Puddle-159 9 subdecoders on 0.005 on Pegasus Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240702_233736-ap4spv1x/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# denim-smoke-166 9 subdecoders on 0.005 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240705_110546-io27epy3/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# hearty-energy-168 9 subdecoders on 0.007 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240705_190726-t13cwqm8/files/AtlasConditionalQVAEv2_atlas_default_130.pth"
# smooth-water-178 25,5,5,5,5 heavy-sided subdecoders on 0.005 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240709_191350-r1fwwnrm/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# rich-glade-179 25,5,5,5,5 heavy-sided subdecoders on 0.01 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240709_192333-p8vm6r9d/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# worthy-vortex-203 no Hierarchical Decoders - 3D Convolutions on 0.005 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240713_101327-d3gqz2n9/files/AtlasConditionalQVAEv2_atlas_default_best.pth"
# jumping-wave-204 no Hierarchical Decoders - 3D Convolutions on 0.01 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240713_101633-v3azs68p/files/AtlasConditionalQVAEv2_atlas_default_150.pth"
# leafy-spaceship-215 no Hierarchical Decoders - 1 Extra 3D Convolutions with Center Padding on 0.005 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240716_092220-6rii3gnx/files/AtlasConditionalQVAE3D_atlas_default_best.pth"
# robust-serenity-216 no Hierarchical Decoders - 1 Extra 3D Convolutions with Center Padding on 0.01 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240716_092439-ja6l8prq/files/AtlasConditionalQVAE3D_atlas_default_150.pth"
# mild-pond-223 no Hierarchical Decoders - 1 Extra 3D Convolutions with Center Padding on 0.01 on Pegasus 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240717_230102-6jq8dbkm/files/AtlasConditionalQVAE3D_atlas_default_best.pth"
# kind-meadow-230 9 Sub Decoders - 1 Extra 3D Convolutions with Center Padding on 0.005 on Zephyr Topology
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240722_232630-qs8sl6t1/files/AtlasConditionalQVAE3DHD_atlas_default_100.pth"
# royal-thunder-232 9 Sub Decoders - HD 1 Extra 3D Convolutions with Center Padding on 0.01 on Zephyr 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240722_235537-ws2su3jq/files/AtlasConditionalQVAE3DHD_atlas_default_best.pth"
# firm-field-235 9 Sub Decoders - HD drop pos energy BP 1 Extra 3D Convolutions with Center Padding on 0.01 on Zephyr 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240726_185231-sqlc60h9/files/AtlasConditionalQVAE3DHD_atlas_default_130.pth"
# playful-sun-263 9 Sub Decoders - HD dropout 0.2 BP 1 Extra 3D Convolutions with Center Padding on 0.01 on Zephyr 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240808_083125-kdoniahe/files/AtlasConditionalQVAE3DHD_atlas_default_best.pth"
# playful-sun-265 9 Sub Decoders - HD lr 0.001, weight decay 0.001, dropout 0.2 BP 1 Extra 3D Convolutions with Center Padding on 0.01 on Zephyr 
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20240809_192828-bk3n30hp/files/AtlasConditionalQVAE3DHD_atlas_default_best.pth"
# ethereal-meadow-666 UNET QVAE with size 7 skip connections
# run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20250315_231215-nt1vh19r/files/AtlasConditionalQVAE3DHD_atlas_default_best.pth"
# good-haze-675 UNET QVAE (skipcon configurations) with size 15 skip connections
run_path = "/fast_scratch_1/caloqvae/luian1/wandb/run-20250326_172648-1jifk4nt/files/AtlasConditionalQVAE3DHD_atlas_default_150.pth"


modelname = 'good-haze-675'
datascaled = 'reduced'
with open("/fast_scratch_1/caloqvae/luian1/wandb/run-20250326_172648-1jifk4nt/files/config.yaml", 'r') as file:
    model_config = yaml.safe_load(file)
    R = json.loads(model_config["_content"]["value"]['engine'].replace("'", "\""))['r_param']
    reducedata = True if model_config["_content"]["value"]["reducedata"] == 'True' else False
    scaled=False
    
arch = config['model']['model_type']
part = config['data']['particle_type']
print(arch)
print(part)
print(scaled, reducedata)


# load_state(model, run_path, 'cuda:{0}'.format(cfg.gpu_list[0]))
# load_state(model, run_path, dev)
modelCreator.load_state(run_path, dev)
engine.model.eval();

[18:46:31.410] INFO   models.modelCreator                               Loading state


AtlasConditionalQVAE3DHD
electron-ds2
False True


[18:46:32.359] INFO   models.modelCreator                               Loading weights from file : /fast_scratch_1/caloqvae/luian1/wandb/run-20250326_172648-1jifk4nt/files/AtlasConditionalQVAE3DHD_atlas_default_150.pth


Loading weights for module =  _activation_fct
Loading weights for module =  _bce_loss
Loading weights for module =  _energy_activation_fct
Loading weights for module =  _hit_activation_fct
Loading weights for module =  _output_loss
Loading weights for module =  _hit_loss
Loading weights for module =  _hit_smoothing_dist_mod
Loading weights for module =  _inference_energy_activation_fct
Loading weights for module =  encoder
Loading weights for module =  prior
Loading weights for module =  decoder


In [14]:

partition_size=config.model.n_latent_nodes_per_p
encoded_data = []
energy_encoded_data = []
n_samples4_qpu = 200

# encoded_data_rdm = []
# energy_encoded_data_rdm = []
engine.model.eval()
# engine_2.model.eval()
with torch.no_grad():
    for xx in val_loader:
    # for xx in train_loader:
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        #################################################
        # true_energy = true_energy[:n_samples4_qpu,:]
        # in_data = in_data[:n_samples4_qpu,:]
        #################################################
        if reducedata:
            in_data = engine._reduce(in_data, true_energy, R=R)
        # enIn = torch.cat((in_data, true_energy), dim=1)
        # beta, post_logits, post_samples = engine.model.encoder(enIn, False)
        beta, post_logits, post_samples = engine.model.encoder(in_data, true_energy, False)
        post_samples = torch.cat(post_samples, 1)
        post_samples_energy = engine.model.stater.energy_samples(post_samples[:,0:partition_size], post_samples[:,partition_size:2*partition_size], 
                                                 post_samples[:,2*partition_size:3*partition_size], post_samples[:,3*partition_size:4*partition_size], 1.0 )
        encoded_data.append(post_samples.detach().cpu())
        energy_encoded_data.append(post_samples_energy.detach().cpu())
        
        # #Rdm model
        # # enIn = torch.cat((in_data, true_energy), dim=1)
        # # beta, post_logits, post_samples = engine.model.encoder(enIn, False)
        # beta, post_logits, post_samples = engine_2.model.encoder(in_data, true_energy, False)
        # post_samples = torch.cat(post_samples, 1)
        # post_samples_energy = engine_2.model.stater.energy_samples(post_samples[:,0:partition_size], post_samples[:,partition_size:2*partition_size], 
        #                                          post_samples[:,2*partition_size:3*partition_size], post_samples[:,3*partition_size:4*partition_size], 1.0 )
        # encoded_data_rdm.append(post_samples.detach().cpu())
        # energy_encoded_data_rdm.append(post_samples_energy.detach().cpu())

encoded_data = torch.cat(encoded_data, dim=0)
energy_encoded_data = torch.cat(energy_encoded_data, dim=0)
        
# encoded_data_rdm = torch.cat(encoded_data_rdm, dim=0)
# energy_encoded_data_rdm = torch.cat(energy_encoded_data_rdm, dim=0)

p1,p2,p3,p4 = post_samples[:,0:partition_size], post_samples[:,partition_size:2*partition_size], \
                                                 post_samples[:,2*partition_size:3*partition_size], post_samples[:,3*partition_size:4*partition_size]

energy_rbm_data = []
rbm_data = []
# energy_rbm_rdm_data = []
with torch.no_grad():
    # for i in range(10):
    for xx in val_loader:
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        ##################################################
        # true_energy = true_energy[:n_samples4_qpu,:]
        # in_data = in_data[:n_samples4_qpu,:]
        ##################################################
        # if i == 0:
            # p1, p2, p3, p4 = engine.model.stater.block_gibbs_sampling_ais(1.0)
        # else:
            # p1, p2, p3, p4 = engine.model.stater.block_gibbs_sampling_ais(1.0, p1, p2, p3, p4)
        engine._model.sampler._batch_size = true_energy.shape[0]
        if True:
            u = engine.model.encoder.binary_energy(true_energy).to(dtype=torch.float32)
            p1, p2, p3, p4 = engine.model.sampler.block_gibbs_sampling_cond(u)
        else:
            p1, p2, p3, p4 = engine.model.sampler.block_gibbs_sampling()
        rbm_data.append(torch.cat((p1,p2,p3,p4),1))
        rbm_samples_energy = engine.model.stater.energy_samples(p1, p2, p3, p4, 1.0)
        energy_rbm_data.append(rbm_samples_energy.detach().cpu())
        engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
        
        # if i == 0:
        #     p1_r, p2_r, p3_r, p4_r = engine_2.model.stater.block_gibbs_sampling_ais(1.0)
        # else:
        #     p1_r, p2_r, p3_r, p4_r = engine_2.model.stater.block_gibbs_sampling_ais(1.0, p1_r, p2_r, p3_r, p4_r)
        # rbm_rdm_samples_energy = engine_2.model.stater.energy_samples(p1_r, p2_r, p3_r, p4_r, 1.0)
        # energy_rbm_rdm_data.append(rbm_rdm_samples_energy.detach().cpu())
    
energy_rbm_data = torch.cat(energy_rbm_data, dim=0)
rbm_data = torch.cat(rbm_data,0)
# energy_rbm_rdm_data = torch.cat(energy_rbm_rdm_data, dim=0)

In [15]:
engine.model.sampling_time_qpu
engine.model.sampling_time_gpu

[]

In [16]:
# en_labels = []

xtarget_samples = []
xrecon_samples = []
xgen_samples = []
xgen_samples_qpu = []
n_samples4_qpu = 200

# xrecon_samples_2 = []

# labelstarget_samples = []
# labelsrecon_samples = []
entarget_samples = []
with torch.no_grad():
    for xx in val_loader:
    # for xx in train_loader:
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        ###############################################
        # true_energy = true_energy[:n_samples4_qpu,:]
        # in_data = in_data[:n_samples4_qpu,:]
        ##############################################
        # print(in_data.shape)
        if reducedata:
            in_data = engine._reduce(in_data, true_energy, R=R)
        fwd_output = engine.model((in_data, true_energy), False)
        if reducedata:
            in_data = engine._reduceinv(in_data, true_energy, R=R)
            recon_data = engine._reduceinv(fwd_output.output_activations, true_energy, R=R)
            engine._model.sampler._batch_size = true_energy.shape[0]
            if True:
                sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy, num_samples=1, thrsh=30, beta=1/beta0)
            else:
                sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
            sample_data_qpu = engine._reduceinv(sample_data_qpu, sample_energies_qpu, R=R)
        elif scaled:
            in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
            recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
            # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_just_act.detach().cpu().numpy()))
            # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_energy_shift.detach().cpu().numpy()))
            engine._model.sampler._batch_size = true_energy.shape[0]
            
            if True:
                sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy[:100,:], num_samples=1, thrsh=30, beta=1/beta0)
            else:
                sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
                # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
            # sample_data_qpu = torch.tensor(engine._data_mgr.inv_transform(sample_data_qpu.detach().cpu().numpy()))
        else:
            in_data = in_data.detach().cpu()*1000
            recon_data = fwd_output.output_activations.detach().cpu()*1000
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            # sample_energies, sample_data = engine._model.generate_samples(num_samples=2048)
            sample_data = sample_data.detach().cpu()*1000


        xtarget_samples.append(in_data.detach().cpu())
        xrecon_samples.append( recon_data.detach().cpu())
        xgen_samples.append( sample_data.detach().cpu())
        # xgen_samples_qpu.append( sample_data_qpu.detach().cpu())
        entarget_samples.append(true_energy.detach().cpu())

        # xrecon_samples_2.append( recon_data_2.detach().cpu())
    
    
xtarget_samples = torch.cat(xtarget_samples, dim=0)
xrecon_samples = torch.cat(xrecon_samples, dim=0)
xgen_samples = torch.cat(xgen_samples, dim=0)
xgen_samples_qpu = torch.cat(xgen_samples_qpu, dim=0)
entarget_samples = torch.cat(entarget_samples, dim=0)

# xrecon_samples_2 = torch.cat(xrecon_samples_2, dim=0)

NameError: name 'sample_data_qpu' is not defined

In [ ]:
# FOR QPU Samples Generation
# import time
# # en_labels = []

# xtarget_samples = []
# xrecon_samples = []
# xgen_samples = []
# xgen_samples_qpu = []
# n_samples4_qpu = 1000

# # xrecon_samples_2 = []

# # labelstarget_samples = []
# # labelsrecon_samples = []
# entarget_samples = []
# with torch.no_grad():
#     for xx in val_loader:
#     # for xx in train_loader:
#         in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
#         ###############################################
#         # true_energy = true_energy[:n_samples4_qpu,:]
#         # in_data = in_data[:n_samples4_qpu,:]
#         ##############################################
#         # print(in_data.shape)
#         if reducedata:
#             in_data = engine._reduce(in_data, true_energy, R=R)
#         fwd_output = engine.model((in_data, true_energy), False)
#         if reducedata:
#             in_data = engine._reduceinv(in_data, true_energy, R=R)
#             recon_data = engine._reduceinv(fwd_output.output_activations, true_energy, R=R)
#             engine._model.sampler._batch_size = true_energy.shape[0]
#             if True:
#                 sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
#                 sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy, num_samples=1, thrsh=15, beta=1/beta0)
#             else:
#                 sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
#                 # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
#             engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
#             sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
#             sample_data_qpu = engine._reduceinv(sample_data_qpu, sample_energies_qpu, R=R)
#         elif scaled:
#             in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
#             recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
#             # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_just_act.detach().cpu().numpy()))
#             # recon_data_2 = torch.tensor(engine._data_mgr.inv_transform(fwd_energy_shift.detach().cpu().numpy()))
#             engine._model.sampler._batch_size = true_energy.shape[0]
            
#             if True:
#                 sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
#                 # sample_energies_qpu, sample_data_qpu = engine.model.generate_samples_qpu_cond(true_energy=true_energy[:100,:], num_samples=1, thrsh=30, beta=1/beta0)
#             else:
#                 sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
#                 # sample_energies_qpu, sample_data_qpu = engine._model.generate_samples_qpu(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True, beta=1/beta0)
#             engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
#             sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
#             # sample_data_qpu = torch.tensor(engine._data_mgr.inv_transform(sample_data_qpu.detach().cpu().numpy()))
#         else:
#             in_data = in_data.detach().cpu()*1000
#             recon_data = fwd_output.output_activations.detach().cpu()*1000
#             engine._model.sampler._batch_size = true_energy.shape[0]
#             sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
#             engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
#             # sample_energies, sample_data = engine._model.generate_samples(num_samples=2048)
#             sample_data = sample_data.detach().cpu()*1000


#         xtarget_samples.append(in_data.detach().cpu())
#         xrecon_samples.append( recon_data.detach().cpu())
#         xgen_samples.append( sample_data.detach().cpu())
#         xgen_samples_qpu.append( sample_data_qpu.detach().cpu())
#         entarget_samples.append(true_energy.detach().cpu())

#         time.sleep(2.5)

#         # xrecon_samples_2.append( recon_data_2.detach().cpu())
    
    
# xtarget_samples = torch.cat(xtarget_samples, dim=0)
# xrecon_samples = torch.cat(xrecon_samples, dim=0)
# xgen_samples = torch.cat(xgen_samples, dim=0)
# xgen_samples_qpu = torch.cat(xgen_samples_qpu, dim=0)
# entarget_samples = torch.cat(entarget_samples, dim=0)

# # xrecon_samples_2 = torch.cat(xrecon_samples_2, dim=0)

In [ ]:
beta, beta_list, rbm_energy_list, dwave_energy_list, thrsh_met = engine.model.find_beta(num_reads=1024, beta_init=8.1, lr=0.01, num_epochs = 30, delta = 4.0, method = 2, 
                                                                                        TOL=True, const = 1.0, adaptive = True)
beta0 = beta

In [ ]:
plt.scatter(range(len(beta_list)), beta_list, linewidth=2.5, color="b" )
plt.plot(range(len(beta_list)), beta_list, linewidth=1.5, color="b" )
plt.xlabel("Iterations", fontsize=15)
plt.ylabel("Estimated $β_{QA}$", fontsize=15)
plt.legend([f'Chip {engine.model._qpu_sampler.properties["chip_id"]}'], fontsize=15)
# plt.title(f'{ds[part]}')
plt.grid("True")
plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/beta_QA_{engine.model._qpu_sampler.properties["chip_id"]}_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
QA = "$β_{QA}$"
Hoffset = -(sum([engine.model.prior.bias_dict[key].sum().detach().cpu().item() for key in engine.model.prior.bias_dict.keys()])/2 
            + sum([engine.model.prior.weight_dict[key].sum().detach().cpu().item() for key in engine.model.prior.weight_dict.keys()])/4)
print(Hoffset)

minVal, maxVal = min(dwave_energy_list[-1] + Hoffset), max(dwave_energy_list[-1] + Hoffset)
# minVal, maxVal = min(energy_dwave.detach().cpu().numpy()+ Hoffset/2), max(energy_dwave.detach().cpu().numpy()+ Hoffset/2)
binwidth = (maxVal-minVal)/10

plt.figure(figsize=(7,6))
# plt.hist(energy_encoded_data.numpy(), bins=30, linewidth=2.5, color="b", density=True, log=True)
plt.hist(rbm_energy_list[-1] + Hoffset, density=True, color="m", alpha=0.5, bins=np.arange(minVal, maxVal + binwidth, binwidth))
plt.hist(dwave_energy_list[-1] + Hoffset, density=True, color="#4B0082", alpha=0.5, bins=np.arange(minVal, maxVal + binwidth, binwidth))
plt.xlabel("RBM Energy", fontsize=15)
plt.ylabel("PDF", fontsize=15)
plt.legend(["Classical samples", "QPU samples"], fontsize=14)
# plt.title(f'ln(Z)[AIS] = {np.round(Zais)}, ln(Z)[RAIS] = {np.round(Zrais)}')
plt.figtext(0.735, 0.6, f'Est. {QA} = {np.round(beta0, 2)}', ha='center', va='top', fontsize=17, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=1'))
plt.subplots_adjust(bottom=0.2)
plt.grid("True")
plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/Ising_energy_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()
print(len(rbm_energy_list[-1]))
print(len(dwave_energy_list[-1]))

In [ ]:
# save_dir = '/home/luian1/CaloQVAE/data/QPUSamples/'
# print(xtarget_samples.shape)
# torch.save(xgen_samples_qpu, save_dir + modelname + 'qpusamples.pt')

In [ ]:
print(post_samples.shape)

In [ ]:
import time
decoder_time = []
for i in range(1000):
    # start = time.process_time()
    start = time.time()
    output_hits, output_activations = engine.model.decoder(post_samples, true_energy)
    torch.cuda.current_stream().synchronize()
    t1 = time.time()
    decoder_time.append(t1 - start)
    # decoder_time.append(time.process_time() - start)

In [ ]:
xtarget_samples_orig = xtarget_samples
entarget_samples_orig = entarget_samples
print(xtarget_samples, xtarget_samples.shape)

In [ ]:
print(decoder_time)
plt.plot(decoder_time)

In [ ]:
import h5py

# New test data
test_data = h5py.File('/fast_scratch_1/caloqvae/test_data/dataset_2_2.hdf5', 'r')
print(test_data.keys())
print(test_data['incident_energies'].shape)
print(test_data['showers'].shape)

In [ ]:
data_showers = (np.array(test_data['showers']))
data_showers = torch.tensor(data_showers)
print(data_showers, data_showers.shape)

In [ ]:
# Data from good model
# gen_samps = h5py.File('/fast_scratch_1/caloqvae/syn_data/wise-tree-1429.hdf5', 'r')
gen_samps_orig = h5py.File('/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_denim-smoke-166en130.hdf5', 'r')
# gen_samps = h5py.File('/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_dutiful-silence-108v2.hdf5', 'r')
print(len(gen_samps))
print(gen_samps.keys())
print((gen_samps['incidence energy']).shape)
print((gen_samps['showers']).shape)
print(xtarget_samples.numpy().shape)
factor = (xtarget_samples.numpy().shape)[0]/((gen_samps['showers']).shape)[0]
print(factor)

In [ ]:
entarget_samples = torch.tensor(gen_samps['incidence energy'])
# xgen_samples = torch.tensor(gen_samps_orig['showers'])
xgen_samples_1429 = torch.tensor(gen_samps['showers'])
print(xgen_samples, xgen_samples.shape)
print(xgen_samples_1429, xgen_samples_1429.shape)
print(np.histogram((xgen_samples_1429 == 0).sum(dim=1) / xtarget_samples.shape[1], bins=20))

In [ ]:
xtarget_samples = data_showers
print("Ground Truth: ", xtarget_samples.shape)
print("Recon Size: ", xrecon_samples.shape)
print("Samples Size: ", xgen_samples.shape)
print("Comparison Samples Size: ", xgen_samples_1429.shape)

In [ ]:
plt.figure(figsize=(8,6))
plt.hist(energy_encoded_data.numpy(), bins=30, linewidth=2.5, color="b", density=True, log=True)
plt.hist(energy_rbm_data.numpy(), bins=30, color="orange", density=True, fc=(1, 0, 1, 0.5), histtype='step', linewidth=1.5)
# plt.hist(energy_rbm_rdm_data.numpy(), bins=20, linewidth=2.5, color="cyan", density=True, fc=(0.5, 1.0, 0.5, 0.8))
# plt.hist(energy_encoded_data_rdm.numpy(), bins=70, linewidth=2.5, color="r", density=True)

plt.xlabel("RBM Energy", fontsize=15)
plt.ylabel("PDF", fontsize=15)
plt.legend(["Trained RBM w/ Encoded Data", "Trained RBM w/ Gibbs sampled data", "Random RBM w/ Gibbs sampled data", "Random RBM w/ Init Encoded Data"], fontsize=17, loc='lower left')
# plt.title(f'ln(Z)[AIS] = {np.round(Zais)}, ln(Z)[RAIS] = {np.round(Zrais)}')
# plt.title(f'LL(trained) = {np.round(-energy_encoded_data.mean() - lnZais)}, LL(Rdm) = {np.round(-energy_encoded_data_rdm.mean() - lnZrais_rdm)} \n \
        # LL(trained RBM data) = {np.round(-energy_rbm_data.mean() - lnZais)}, LL(Rdm RBM data) = {np.round(-energy_rbm_rdm_data.mean() - lnZrais_rdm)}')
plt.grid("True")
# plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/RBM_energy_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
hfont = {'fontname':'Helvetica'}
ds = {'electron-ds2':'Dataset 2', 'pion1':'Dataset 1: π'}

In [ ]:
minVal, maxVal = min(((xtarget_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy()), max(((xtarget_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy())
binwidth = (maxVal-minVal)/30

plt.hist(((xtarget_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
plt.hist(((xrecon_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="c")
plt.hist(((xgen_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
# plt.hist((xgen_samples_1429 == 0).sum(dim=1) / xtarget_samples.shape[1], bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
plt.hist((xgen_samples_qpu == 0).sum(dim=1) / xtarget_samples.shape[1], bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")

plt.xlabel("Sparsity Index")
plt.ylabel("Histogram")
plt.legend(["Ground Truth", "Reconstruction", "Classic Samples", "QPU Samples"])
# plt.title(f'{ds[part]}')
plt.grid("True")
plt.show()

In [ ]:
import os
HLF_1_electron.relevantLayers = [0,5,10,15,20,25,30,35,40,45]

# Create lists for the histogram values
target_counts, recon_counts, sample_counts = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)
target_bins, recon_bins, sample_bins = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)

plt.figure(figsize=(8,6))
# Create a grid of subplots
fig, axes = plt.subplots(3,3, figsize=(12, 12), sharey=True, sharex=True, tight_layout=True)
fig.text(0.5, -0.01, 'Sparsity Index', ha='center', fontsize=15)
fig.text(-0.01, 0.5, 'Histogram', va='center', rotation='vertical', fontsize=15)

# Iterate through the columns of X and plot histograms
for i,_ in enumerate(HLF_1_electron.relevantLayers[:-1]):
    row_index = i // 3  # Determine the row index
    col_index = i % 3   # Determine the column index
    
    ax = axes[row_index, col_index]  # Get the current subplot
    
    # Plot histogram for the current column
    idx = HLF_1_electron.relevantLayers[i+1]*9*16
    idxPrev = (HLF_1_electron.relevantLayers[i])*9*16
    l = idx - idxPrev
    print(idx, idxPrev)
    minVal, maxVal = min(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy()), max(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy())
    binwidth = (maxVal-minVal)/30
    
    # Store the values of the histogram with the same normalization and bin sizes
    target_counts[i], target_bins[i] = np.histogram(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
    recon_counts[i], recon_bins[i] = np.histogram(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
    sample_counts[i], sample_bins[i] = np.histogram(((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
    ax.hist(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(),  bins=np.arange(minVal, maxVal+binwidth, binwidth), log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
    ax.hist(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal+binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="c")
    ax.hist(((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal+binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
    # ax.hist(((xgen_samples_1429[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal+binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
    ax.hist(((xgen_samples_qpu[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal+binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
    ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], fontsize=14, loc = 'upper left')
    ax.grid("True")
    
    # Set labels and title for the subplot
    # ax.set_xlabel(f'Column {i + 1}')
    # ax.set_ylabel('Frequency')
    ax.set_title(f'Layers {HLF_1_electron.relevantLayers[i]} to {HLF_1_electron.relevantLayers[i+1]-1}', fontsize=12)

# Adjust layout and display the plots
plt.tight_layout()
os.makedirs(f'/home/luian1/CaloQVAE/figs/{modelname}/sparsity_per_layer_{modelname}_{arch}_{datascaled}_{part}', exist_ok=True)
plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/sparsity_per_layer_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
# HLF_1_electron.relevantLayers = [0,5,10,15,20,25,30,35,40,45]

# # Energy Ranges
# E_right = 50000000000
# E_left = 100000
# tmp = (entarget_samples < E_right) * (entarget_samples > E_left)
# idxEnFilter = (tmp == True).nonzero(as_tuple=True)[0]

# # Create lists for the histogram values
# target_counts, recon_counts, sample_counts = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)
# target_bins, recon_bins, sample_bins = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)

# plt.figure(figsize=(8,6))
# # Create a grid of subplots
# fig, axes = plt.subplots(3,3, figsize=(12, 12), sharey=True, sharex=True, tight_layout=True)
# fig.text(0.5, -0.01, 'Sparsity Index', ha='center', fontsize=15)
# fig.text(-0.01, 0.5, 'Histogram', va='center', rotation='vertical', fontsize=15)

# # Iterate through the columns of X and plot histograms
# for i,_ in enumerate(HLF_1_electron.relevantLayers[:-1]):
#     row_index = i // 3  # Determine the row index
#     col_index = i % 3   # Determine the column index
    
#     ax = axes[row_index, col_index]  # Get the current subplot
    
#     # Plot histogram for the current column
#     idx = HLF_1_electron.relevantLayers[i+1]*9*16
#     idxPrev = (HLF_1_electron.relevantLayers[i])*9*16
#     l = idx - idxPrev
#     minVal, maxVal = min(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy()), max(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy())
#     binwidth = (maxVal-minVal)/30
    
#     # Store the values of the histogram with the same normalization and bin sizes
#     target_counts[i], target_bins[i] = np.histogram(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
#     recon_counts[i], recon_bins[i] = np.histogram(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
#     sample_counts[i], sample_bins[i] = np.histogram(((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
#     ax.hist((((xtarget_samples[idxEnFilter])[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(),  bins=np.arange(minVal, maxVal, binwidth), log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
#     ax.hist((((xrecon_samples[idxEnFilter])[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="c")
#     ax.hist((((xgen_samples[idxEnFilter])[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
#     ax.hist((((xgen_samples_1429[idxEnFilter])[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
#     # ax.hist(((xgen_samples_qpu[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="m", linestyle="dashed")
#     if i == 0:
#         ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], fontsize=14, loc = 'upper left')
#     ax.grid("True")
    
#     # Set labels and title for the subplot
#     # ax.set_xlabel(f'Column {i + 1}')
#     # ax.set_ylabel('Frequency')
#     ax.set_title(f'Layers {HLF_1_electron.relevantLayers[i]} to {HLF_1_electron.relevantLayers[i+1]-1}', fontsize=12)

# # Adjust layout and display the plots
# plt.tight_layout()
# plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/sparsity_per_layer_{modelname}_{arch}_{datascaled}_{part}.png')
# plt.show()

In [ ]:
def KL_div(target, inp):
    KL_met = 0
    for i in range(min(len(inp), len(target))):
        if target[i] == 0:
            KL_met += 0
        else:
            KL_met += target[i] * np.log(abs(target[i]/inp[i]))
    return KL_met / min(len(inp), len(target))

def JSD(P, Q):
    P = np.asarray(P, dtype=np.float)
    Q = np.asarray(Q, dtype=np.float)
    M = 0.5 * (P + Q)
    return 0.5 * (KL_div(P, M) + KL_div(Q, M))

def Hellinger_dist(target, inp):
    Hel_dist = 0
    for i in range(min(len(inp), len(target))):
        Hel_dist += ((target[i]) ** 0.5 - (inp[i]) ** 0.5) ** 2
    return (1 / (2 ** 0.5)) * ((Hel_dist) ** 0.5)

# print(idxPrev, idx)
# print(xtarget_samples.size()) # [# of data cylinders, # voxels per cylinder]
# print((xtarget_samples[:, idxPrev:idx] == 0))
# # l = idx = idxPrev
# print((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1))
# print(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), ((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy().shape)

# plt.plot(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy())
# plt.show()

# print(target_counts[0], recon_counts[0])
# print("KL div b/w target and recon: ", KL_div(target_counts[0], recon_counts[0]))
# print("KL div b/w target and gen: ", KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), ((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy()))

In [ ]:
# Calculate a Jensen-Shannon Divergence (JSD) a smoothed version of KL divergence to handle / 0

layers = np.linspace(0, 44, 45)
JSD_recon = np.zeros(45)
JSD_gen1 = np.zeros(45)
JSD_gen2 = np.zeros(45)

HLD_recon = np.zeros(45)
HLD_gen1 = np.zeros(45)
HLD_gen2 = np.zeros(45)

# Create lists for the histogram values
target_counts, recon_counts, gen_counts, gen_counts_1429 = [0] * 44, [0] * 44, [0] * 44, [0] * 44
target_bins, recon_bins, gen_bins, gen_bins_1429 = [0] * 44, [0] * 44, [0] * 44, [0] * 44

for i in range(44): # of layers in the cylinder - 1
    idx = (i+1)*9*16
    idxPrev = (i)*9*16
    l = idx - idxPrev
    minVal, maxVal = min(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy()), max(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy())
    binwidth = (maxVal-minVal)/30
    target_counts[i], target_bins[i] = np.histogram(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), density=True)
    recon_counts[i], recon_bins[i] = np.histogram(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), density=True)
    gen_counts[i], gen_bins[i] = np.histogram(((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), density=True)
    gen_counts_1429[i], gen_bins_1429[i] = np.histogram(((xgen_samples_1429[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), density=True)

#     gen_counts[i], gen_bins[i] = np.histogram(((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), weights = np.ones_like((xgen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l) * 0.1)
    mid_recon = 0.5 * (target_counts[i] + recon_counts[i])
    mid_gen1 = 0.5 * (target_counts[i] + gen_counts[i])
    mid_gen2 = 0.5 * (target_counts[i] + gen_counts_1429[i])
    
#     print(mid_recon, target_counts[0].tolist())
    
#     print("testing: ", KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))
#     print((0.5 * (KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon) + KL_div(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))))
   
    JSD_recon[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_recon) + KL_div(recon_counts[i].tolist(), mid_recon)))
    JSD_gen1[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen1) + KL_div(gen_counts[i].tolist(), mid_gen1)))
    JSD_gen2[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen2) + KL_div(gen_counts_1429[i].tolist(), mid_gen2)))
    
    HLD_recon[i] = Hellinger_dist(target_counts[i].tolist(), recon_counts[i].tolist())
    HLD_gen1[i] = Hellinger_dist(target_counts[i].tolist(), gen_counts[i].tolist())
    HLD_gen2[i] = Hellinger_dist(target_counts[i].tolist(), gen_counts_1429[i].tolist())

print("Layer Size: ", layers.size)
print("JSD Size: ", len(JSD_recon))
plt.scatter(layers[:-1], np.asarray(JSD_recon[:-1]))
plt.scatter(layers[:-1], np.asarray(JSD_gen1[:-1]))
plt.scatter(layers[:-1], np.asarray(JSD_gen2[:-1]))

plt.xlabel("Layer Number")
plt.ylabel("JSD Divergence Value")
plt.legend(["Target vs Recon", "Target vs Sample 1372", "Target vs Sample 454"])
plt.title("JSD Divergence Comparison per Layer")
plt.grid("True")
plt.show()

In [ ]:
plt.scatter(layers[:-1], np.asarray(HLD_recon[:-1]))
plt.scatter(layers[:-1], np.asarray(HLD_gen1[:-1]))
plt.scatter(layers[:-1], np.asarray(HLD_gen2[:-1]))


plt.xlabel("Layer Number")
plt.ylabel("Hellinger Distance")
plt.legend(["Target vs Recon", "Target vs Sample 1372", "Target vs Sample 454"])
plt.title("Hellinger Distance Comparison per Layer")
plt.grid("True")
plt.show()

In [ ]:
print(entarget_samples)
entarget_samplesnp = entarget_samples.numpy()
ind = np.argsort(entarget_samplesnp, axis = 0).flatten()
print(ind)
print(entarget_samples[ind])
sortxtarget_samples = xtarget_samples[ind]
print(sortxtarget_samples, sortxtarget_samples.shape)
print(max(sortxtarget_samples[0]))

In [ ]:
# test = np.logspace(0, np.log(1000000), 44, base=10)
# print(test)

# idxPrev = test[0]
# print(entarget_samples, entarget_samples.shape)
# for i in range(44):
#     print(len(entarget_samples[:, int(test[i]): int(test[i+1])]))

x=np.zeros([4,4])
x[0][0] = 1
print(x)

samps = np.linspace(0, 44, 12)
print(samps)

In [ ]:
gen_samps = h5py.File('/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_robust-serenity-216en150.hdf5', 'r')
xgen_samples_1429 = torch.tensor(gen_samps['showers'])

In [ ]:
print(xtarget_samples.shape, xgen_samples_1429.shape, entarget_samples.shape)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
samps = np.linspace(0, len(entarget_samples) - 1, 10)

layers = np.linspace(0, 44, 45)
print(layers)
# JSD_recon = np.zeros([44,44])
JSD_gen = np.zeros([45,45])

# HLD_recon = np.zeros([44,44])
HLD_gen = np.zeros([45,45])

# Sort samples
entarget_samplesnp = entarget_samples.numpy()
ind = np.argsort(entarget_samplesnp, axis=0).flatten()
sortentarget_samples = entarget_samples[ind]
sortxtarget_samples = xtarget_samples[ind]
# sortxrecon_samples = xrecon_samples[ind]
sortxgen_samples = xgen_samples_1429[ind]
inc_energies = np.linspace(0, len(entarget_samples), 46).astype(int)

xenergies = (((sortentarget_samples[samps] / 1).flatten()).numpy().astype(int)).tolist()

minVal, maxVal = 0, 1
binwidth = (maxVal-minVal)/20

for j in range(45):
    
    # Create lists for the histogram values
    target_counts, gen_counts = [0] * 45, [0] * 45
    target_bins, gen_bins = [0] * 45, [0] * 45

    sub_target_samples = sortxtarget_samples[inc_energies[j] : inc_energies[j+1]]
    # sub_recon_samples = sortxrecon_samples[inc_energies[j] : inc_energies[j+1]]
    sub_gen_samples = sortxgen_samples[inc_energies[j] : inc_energies[j+1]]

    for i in range(45): # of layers in the cylinder - 1

        idx = (i+1)*9*16
        idxPrev = (i)*9*16
        l = idx - idxPrev
        
        target_counts[i], target_bins[i] = np.histogram(((sub_target_samples[:, idxPrev:idx] == 0).sum(axis=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
        # recon_counts[i], recon_bins[i] = np.histogram(((sub_recon_samples[:, idxPrev:idx] == 0).sum(axis=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth), density=True)
        gen_counts[i], gen_bins[i] = np.histogram(((sub_gen_samples[:, idxPrev:idx] == 0).sum(axis=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
        # mid_recon = 0.5 * (target_counts[i] + recon_counts[i])
        mid_gen = 0.5 * (target_counts[i] + gen_counts[i])

        # JSD_recon[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_recon) + KL_div(recon_counts[i].tolist(), mid_recon)))
        JSD_gen[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen) + KL_div(gen_counts[i].tolist(), mid_gen)))

        # HLD_recon[j][i] = Hellinger_dist(target_counts[i].tolist(), recon_counts[i].tolist())
        HLD_gen[j][i] = Hellinger_dist(target_counts[i].tolist(), gen_counts[i].tolist())

        if i == 0: 
            print(target_counts, len(target_counts))
            print(gen_counts)

# print(JSD_recon)
# print(min(JSD_recon.flatten()))
# print(JSD_recon, JSD_recon.shape)

# Plotting for JSD_gen
fig, ax = plt.subplots(figsize=(8, 6))

cax = ax.imshow(JSD_gen, interpolation='nearest', cmap='viridis')
ax.set_aspect('equal')
ax.set_title("Ground Truth vs Model Prediction (Sparsity Index)")
ax.set_yticks(np.linspace(0, 44, 10))
ax.set_yticklabels([int(x) for x in xenergies])
ax.set_ylabel("Incident Energy Ranges (MeV)")
ax.set_xlabel("Layer Number")

divider = make_axes_locatable(ax)
cax_cbar = divider.append_axes("right", size="5%", pad=0.25)
cbar = fig.colorbar(cax, cax=cax_cbar)
cbar.set_label('JSD Divergence Value')

plt.tight_layout()
plt.show()

In [ ]:
# Plotting for HLD_recon and HLD_gen
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
fig, ax2 = plt.subplots(figsize=(12, 6))

# Plot for HLD_recon
# cax1 = ax1.imshow(HLD_recon, interpolation='nearest', cmap='viridis')
# ax1.set_aspect('equal')
# ax1.set_yticks(np.linspace(0, 43, 10))
# ax1.set_yticklabels([int(x) for x in xenergies])
# ax1.set_ylabel("Incident Energy Ranges")
# ax1.set_xlabel("Layer Number")
# divider1 = make_axes_locatable(ax1)
# cax_cbar1 = divider1.append_axes("right", size="5%", pad=0.25)
# cbar1 = fig.colorbar(cax1, cax=cax_cbar1)
# cbar1.set_label('Hellinger Distance Value')

# Plot for HLD_gen
cax2 = ax2.imshow(HLD_gen, interpolation='nearest', cmap='viridis')
ax2.set_aspect('equal')
ax2.set_yticks(np.linspace(0, 43, 10))
ax2.set_yticklabels([int(x) for x in xenergies])
ax2.set_ylabel("Incident Energy Ranges")
ax2.set_xlabel("Layer Number")
divider2 = make_axes_locatable(ax2)
cax_cbar2 = divider2.append_axes("right", size="5%", pad=0.25)
cbar2 = fig.colorbar(cax2, cax=cax_cbar2)
cbar2.set_label('Hellinger Distance Value')

plt.tight_layout()
plt.show()


In [ ]:
plt.scatter(entarget_samples, ((xtarget_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy())
plt.scatter(entarget_samples, ((xrecon_samples == 0).sum(dim=1)/xtarget_samples.shape[1]).numpy())

plt.xlabel("Incidence Energy (GeV)")
plt.ylabel("Sparsity")
plt.legend(["GT", "Recon", "Samples", "Sample /w QPU"])
plt.title(f'{ds[part]}')
plt.grid("True")
plt.show()

In [ ]:
fig, axes = plt.subplots(3,2, figsize=(8,8), tight_layout=True)
fig.text(0.5, 0.0, 'GT energy per event (GeV)', ha='center')
fig.text(0.5, 1.0, f'{ds[part]}', ha='center', fontsize=12)

axes[0,0].scatter(xtarget_samples.sum(dim=1).numpy()/1000, xrecon_samples.sum(dim=1).numpy()/1000, marker='.', alpha=.5, color="b")
axes[0,0].plot([0,800],[0,800], c='red', lw=2.5, label='y=x')
axes[0,0].set_ylabel("Recon energy per event (GeV)")
axes[0,0].legend()
axes[0,0].grid("True")
axes[0,0].set_yscale('log')
axes[0,0].set_xscale('log')

axes[0,1].scatter(xtarget_samples.sum(dim=1).numpy()/1000, (xtarget_samples.sum(dim=1).numpy() - xrecon_samples.sum(dim=1).numpy())/1000, marker='.', alpha=.5, color="b")
axes[0,1].set_ylabel("Abs error (GeV)")
axes[0,1].set_ylim([-40,40])
# axes[0,1].legend()
# axes[0,1].set_yscale('log')
axes[0,1].grid("True")

axes[1,0].scatter(1/(entarget_samples.numpy()/1000), np.abs(entarget_samples.reshape(-1).numpy() - xtarget_samples.sum(dim=1).numpy())/(entarget_samples.reshape(-1).numpy()+1e-3), marker='.', alpha=.5, color="blue", label="Simulation")
axes[1,0].scatter(1/(entarget_samples.numpy()/1000), np.abs(entarget_samples.reshape(-1).numpy() - xrecon_samples.sum(dim=1).numpy())/(entarget_samples.reshape(-1).numpy()+1e-3), marker='.', alpha=.05, color="red", label="Model")
axes[1,0].plot([1e-3,1e0],np.power([1e-3,1e0],0.25)*np.power(10,2.0), c='orange', lw=2.5, label='slope=0.25', linestyle="dashdot")
axes[1,0].grid("True")
axes[1,0].set_yscale('log')
axes[1,0].set_xscale('log')
axes[1,0].legend()
axes[1,0].set_ylabel("Relative Error")

axes[1,1].scatter(entarget_samples.numpy()/1000, np.abs(entarget_samples.reshape(-1).numpy()/1000 - xtarget_samples.sum(dim=1).numpy()/1000), marker='.', alpha=1, color="blue", label="Simulation")
axes[1,1].scatter(entarget_samples.numpy()/1000, np.abs(entarget_samples.reshape(-1).numpy()/1000 - xrecon_samples.sum(dim=1).numpy()/1000), marker='.', alpha=.2, color="red", label="Model")
axes[1,1].plot([1,800],np.sqrt([1,800]), c='orange', lw=2.5, label='y=sqrt(x)', linestyle="dashdot")
axes[1,1].plot([1,800],[1,800], c='c', lw=2.5, label='y=x', linestyle="dashed")
axes[1,1].grid("True")
axes[1,1].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,1].set_ylabel("Absolute Error")
axes[1,1].legend()


# Merge the first row's axes
gs = axes[2, 0].get_gridspec()
for ax in axes[2, :]:
    ax.remove()
ax_big = fig.add_subplot(gs[2, :])
ax_big.scatter(1/(xtarget_samples.sum(dim=1).numpy()/1000), np.abs(xtarget_samples.sum(dim=1).numpy() - xrecon_samples.sum(dim=1).numpy())/(xtarget_samples.sum(dim=1).numpy()+1e-3), marker='.', alpha=.5, color="blue")
ax_big.plot([1e-3,1e1],np.sqrt([1e-3,1e1])*np.power(1,3.5), c='orange', lw=2.5, label='y=sqrt(x)', linestyle="dashdot")
ax_big.grid("True")
ax_big.set_yscale('log')
ax_big.set_xscale('log')
ax_big.legend()
ax_big.set_ylim([1e-5,1e1])
ax_big.set_yticks([1e-5,1e-4,1e-3,1e-2,1e-1,1e0,1e1])
ax_big.set_ylabel("Relative Recon Error")
plt.show()

In [ ]:
#Energy

In [ ]:
minVal, maxVal = min(xtarget_samples.sum(dim=1).numpy()/1000), max(xtarget_samples.sum(dim=1).numpy()/1000)
binwidth = (maxVal-minVal)/30

plt.hist(xtarget_samples.sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
plt.hist(xrecon_samples.sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="c")
plt.hist(xgen_samples.sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
# plt.hist(xgen_samples_1429.sum(dim=1).numpy()/1000, bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
plt.hist(xgen_samples_qpu.sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
# plt.hist(((xgen_samples_1429 == 0).sum(axis=1)/xtarget_samples.shape[1]).numpy(), bins=20, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")

plt.xlabel("Energy per Event (GeV)")
plt.ylabel("Histogram")
plt.legend(["Ground Truth", "Reconstruction", "Classic Samples", "QPU Samples"])
# plt.title("Energy Histogram")
plt.grid("True")
plt.show()

In [ ]:
HLF_1_electron.relevantLayers = [0,5,10,15,20,25,30,35,40,45]

# Create lists for the histogram values
target_counts, recon_counts, sample_counts = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)
target_bins, recon_bins, sample_bins = [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1), [0] * (len(HLF_1_electron.relevantLayers) - 1)

plt.figure(figsize=(8,6))
# Create a grid of subplots
fig, axes = plt.subplots(3,3, figsize=(12, 12), sharey=True, sharex=False, tight_layout=True)
fig.text(0.5, 0.0, 'Energy per event (GeV)', ha='center', fontsize=15)
fig.text(0.0, 0.5, 'Histogram', va='center', rotation='vertical', fontsize=15)

# Iterate through the columns of X and plot histograms
for i,_ in enumerate(HLF_1_electron.relevantLayers[:-1]):
    row_index = i // 3  # Determine the row index
    col_index = i % 3   # Determine the column index
    
    ax = axes[row_index, col_index]  # Get the current subplot

    minVal, maxVal = min(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000), max(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000)
    binwidth = (maxVal-minVal)/30
    
    # Plot histogram for the current column
    idx = HLF_1_electron.relevantLayers[i+1]*9*16
    idxPrev = (HLF_1_electron.relevantLayers[i])*9*16
    l = idx - idxPrev
    
    # Store the values of the histogram with the same normalization and bin sizes
    target_counts[i], target_bins[i] = np.histogram(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20)
    recon_counts[i], recon_bins[i] = np.histogram(xrecon_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20)
    sample_counts[i], sample_bins[i] = np.histogram(xgen_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20)
    
    ax.hist(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000,  bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
    ax.hist(xrecon_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="c")
    ax.hist(xgen_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
    # ax.hist(xgen_samples_1429[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
    ax.hist(xgen_samples_qpu[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=np.arange(minVal, maxVal + binwidth, binwidth), log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")

    if i == 0:
        ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], fontsize=15, loc='upper right')
    ax.grid("True")
    
    # Set labels and title for the subplot
    # ax.set_xlabel(f'Column {i + 1}')
    # ax.set_ylabel('Frequency')
    ax.set_title(f'Layers {HLF_1_electron.relevantLayers[i]} to {HLF_1_electron.relevantLayers[i+1]-1}')

# Adjust layout and display the plots
plt.tight_layout()
plt.savefig(f'/home/luian1/CaloQVAE/figs/{modelname}/sparsity_per_layer_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()


In [ ]:
print(xtarget_samples.shape)

E_right = 12000 # MeV
E_left = 10000 # MeV
tmp = (entarget_samples < E_right) * (entarget_samples > E_left)
idxEnFilter = (tmp == True).nonzero(as_tuple=True)[0]
print(idxEnFilter.shape)
print(xtarget_samples, xtarget_samples.shape)
# print(entarget_samples[idxEnFilter])
print(xtarget_samples[idxEnFilter], xtarget_samples[idxEnFilter].shape)
len(idxEnFilter)

In [ ]:
xtarget_samples = data_showers
print(xtarget_samples, xtarget_samples.shape)
print(xgen_samples_1429, xgen_samples_1429.shape)
print(entarget_samples, entarget_samples.shape)

In [ ]:
print(xrecon_samples, xrecon_samples.shape)

In [ ]:
# Calculate a Jensen-Shannon Divergence (JSD) a smoothed version of KL divergence to handle / 0

layers = np.linspace(0, 44, 45)
JSD_recon = np.zeros(45)
JSD_gen1 = np.zeros(45)
JSD_gen2 = np.zeros(45)

HLD_recon = np.zeros(45)
HLD_gen1 = np.zeros(45)
HLD_gen2 = np.zeros(45)

# Create lists for the histogram values
target_counts, recon_counts, gen_counts, gen_counts_1429 = [0] * 44, [0] * 44, [0] * 44, [0] * 44
target_bins, recon_bins, gen_bins, gen_bins_1429 = [0] * 44, [0] * 44, [0] * 44, [0] * 44

# Energy Ranges
E_right = 5000000000
E_left = 0
tmp = (entarget_samples < E_right) * (entarget_samples > E_left)
idxEnFilter = (tmp == True).nonzero(as_tuple=True)[0]

# Energy filtered samples
print(xtarget_samples)
print(xtarget_samples.shape, idxEnFilter.shape)
# xtarget_samples, xrecon_samples, gen_samples = xtarget_samples[idxEnFilter], xrecon_samples[idxEnFilter], xgen_samples[idxEnFilter] # this line creates problems when running it again 

for i in range(44): # of layers in the cylinder - 1
    idx = (i+1)*9*16
    idxPrev = (i)*9*16
    l = idx - idxPrev
    
    # target_counts[i], target_bins[i] = np.histogram((xtarget_samples[idxEnFilter])[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20, density=True)
    # recon_counts[i], recon_bins[i] = np.histogram((xrecon_samples[idxEnFilter < xrecon_samples.shape[0]])[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20,density=True)
    # gen_counts[i], gen_bins[i] = np.histogram((xgen_samples[idxEnFilter])[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20, density=True)
    # gen_counts_1429[i], gen_bins_1429[i] = np.histogram((xgen_samples[idxEnFilter])[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20, density=True)

    target_counts[i], target_bins[i] = np.histogram(((xtarget_samples)[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20, density=True)
    recon_counts[i], recon_bins[i] = np.histogram(((xrecon_samples)[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20,density=True)
    gen_counts[i], gen_bins[i] = np.histogram(((xgen_samples)[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20, density=True)
    gen_counts_1429[i], gen_bins_1429[i] = np.histogram(((xgen_samples_1429)[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20, density=True)
    
    mid_recon = 0.5 * (target_counts[i] + recon_counts[i])
    mid_gen1 = 0.5 * (target_counts[i] + gen_counts[i])
    mid_gen2 = 0.5 * (target_counts[i] + gen_counts_1429[i])
    
#     print(mid_recon, target_counts[0].tolist())
    
#     print("testing: ", KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))
#     print((0.5 * (KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon) + KL_div(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))))
   
    JSD_recon[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_recon) + KL_div(recon_counts[i].tolist(), mid_recon)))
    JSD_gen1[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen1) + KL_div(gen_counts[i].tolist(), mid_gen1)))
    JSD_gen2[i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen2) + KL_div(gen_counts_1429[i].tolist(), mid_gen2)))
    
    HLD_recon[i] = Hellinger_dist(target_counts[i].tolist(), recon_counts[i].tolist())
    HLD_gen1[i] = Hellinger_dist(target_counts[i].tolist(), gen_counts[i].tolist())
    HLD_gen2[i] = Hellinger_dist(target_counts[i].tolist(), gen_counts_1429[i].tolist())

print("Layer Size: ", layers.size)
print("JSD Size: ", len(JSD_recon))
plt.scatter(layers[:-1], np.asarray(JSD_recon[:-1]))
plt.scatter(layers[:-1], np.asarray(JSD_gen1[:-1]))
plt.scatter(layers[:-1], np.asarray(JSD_gen2[:-1]))

plt.yscale('log')

plt.xlabel("Layer Number")
plt.ylabel("JSD Divergence Value")
plt.legend(["Target vs Recon", "Target vs Sample 1372", "Target vs Sample 454"])
plt.title("JSD Divergence Comparison per Layer")
plt.grid("True")
plt.show()

In [ ]:
plt.scatter(layers[:-1], np.asarray(HLD_recon[:-1]))
plt.scatter(layers[:-1], np.asarray(HLD_gen1[:-1]))
plt.scatter(layers[:-1], np.asarray(HLD_gen2[:-1]))

plt.yscale('log')

plt.xlabel("Layer Number")
plt.ylabel("Hellinger Distance")
plt.legend(["Target vs Recon", "Target vs Sample 1372", "Target vs Sample 454"])
plt.title("Hellinger Distance Comparison per Layer")
plt.grid("True")
plt.show()

In [ ]:
# Calculate a Jensen-Shannon Divergence (JSD) a smoothed version of KL divergence to handle / 0
from mpl_toolkits.axes_grid1 import make_axes_locatable

samps = np.linspace(0, len(entarget_samples_orig) - 1, 10)
print(entarget_samples_orig, entarget_samples_orig.shape)

layers = np.linspace(0, 44, 45)
JSD_recon = np.zeros([45,45])
JSD_gen = np.zeros([45,45])

HLD_recon = np.zeros([45,45])
HLD_gen = np.zeros([45,45])

# Sort samples
entarget_samplesnp = entarget_samples_orig.numpy()
ind = np.argsort(entarget_samplesnp, axis = 0).flatten()
sortentarget_samples = entarget_samples_orig[ind]
sortxtarget_samples = xtarget_samples_orig[ind]
sortxrecon_samples = xrecon_samples[ind]
sortxgen_samples = xgen_samples[ind]
inc_energies = np.linspace(0, len(entarget_samples_orig), 46).astype(int)

xenergies = (((sortentarget_samples[samps] / 1).flatten()).numpy().astype(int)).tolist()

for j in range(45):
    
    # Create lists for the histogram values
    target_counts, recon_counts, gen_counts = [0] * 45, [0] * 45, [0] * 45
    target_bins, recon_bins, gen_bins = [0] * 45, [0] * 45, [0] * 45

    
    sub_target_samples = sortxtarget_samples[inc_energies[j] : inc_energies[j+1]]
    sub_recon_samples = sortxrecon_samples[inc_energies[j] : inc_energies[j+1]]
    sub_gen_samples = sortxgen_samples[inc_energies[j] : inc_energies[j+1]]
    for i in range(45): # of layers in the cylinder - 1
#         print(sub_target_samples, sub_target_samples.shape)
        idx = (i+1)*9*16
        idxPrev = (i)*9*16
        l = idx - idxPrev
#         target_counts[i], target_bins[i] = np.histogram(((sub_target_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
#         recon_counts[i], recon_bins[i] = np.histogram(((sub_recon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
#         gen_counts[i], gen_bins[i] = np.histogram(((sub_gen_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), bins=np.arange(minVal, maxVal + binwidth, binwidth))
        
        target_counts[i], target_bins[i] = np.histogram((sub_target_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)
        recon_counts[i], recon_bins[i] = np.histogram((sub_recon_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)
        gen_counts[i], gen_bins[i] = np.histogram((sub_gen_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)

        
        mid_recon = 0.5 * (target_counts[i] + recon_counts[i])
        mid_gen = 0.5 * (target_counts[i] + gen_counts[i])

    #     print(mid_recon, target_counts[0].tolist())

    #     print("testing: ", KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))
    #     print((0.5 * (KL_div(((xtarget_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon) + KL_div(((xrecon_samples[:, idxPrev:idx] == 0).sum(dim=1)/l).numpy(), mid_recon))))

        JSD_recon[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_recon) + KL_div(recon_counts[i].tolist(), mid_recon)))
        JSD_gen[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen) + KL_div(gen_counts[i].tolist(), mid_gen)))

        HLD_recon[j][i] = Hellinger_dist(target_counts[i].tolist(), recon_counts[i].tolist())
        HLD_gen[j][i] = Hellinger_dist(target_counts[i].tolist(), gen_counts[i].tolist())

    if j == 43:
        print(target_counts, len(target_counts))
        
# print(JSD_recon)
# print(min(JSD_recon.flatten()))
print(JSD_recon, JSD_recon.shape)

# Plotting for JSD_recon and JSD_gen
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Plot for JSD_recon
cax1 = ax1.imshow(JSD_recon, interpolation='nearest', cmap='viridis')
ax1.set_aspect('equal')
ax1.set_title('Target vs Recon')
ax1.set_yticks(np.linspace(0, 43, 10))
ax1.set_yticklabels([int(x) for x in xenergies])
ax1.set_ylabel("Incident Energy Ranges")
ax1.set_xlabel("Layer Number")
divider1 = make_axes_locatable(ax1)
cax_cbar1 = divider1.append_axes("right", size="5%", pad=0.25)
cbar1 = fig.colorbar(cax1, cax=cax_cbar1)
cbar1.set_label('JSD Divergence Value')

# Plot for JSD_gen
cax2 = ax2.imshow(JSD_gen, interpolation='nearest', cmap='viridis')
ax2.set_aspect('equal')
ax2.set_title('Target vs Sample')
ax2.set_yticks(np.linspace(0, 43, 10))
ax2.set_yticklabels([int(x) for x in xenergies])
ax2.set_ylabel("Incident Energy Ranges")
ax2.set_xlabel("Layer Number")
divider2 = make_axes_locatable(ax2)
cax_cbar2 = divider2.append_axes("right", size="5%", pad=0.25)
cbar2 = fig.colorbar(cax2, cax=cax_cbar2)
cbar2.set_label('JSD Divergence Value')

plt.tight_layout()
plt.show()

In [ ]:
print(entarget_samples, entarget_samples.shape)
print(xtarget_samples, xtarget_samples.shape)
print(xgen_samples_1429, xgen_samples_1429.shape)

In [ ]:
samps = np.linspace(0, len(entarget_samples) - 1, 10)

layers = np.linspace(0, 44, 45)
# JSD_recon = np.zeros([44,44])
JSD_gen = np.zeros([45,45])

# HLD_recon = np.zeros([44,44])
HLD_gen = np.zeros([45,45])

# Sort samples
entarget_samplesnp = entarget_samples.numpy()
ind = np.argsort(entarget_samplesnp, axis=0).flatten()
sortentarget_samples = entarget_samples[ind]
sortxtarget_samples = xtarget_samples[ind]
# sortxrecon_samples = xrecon_samples[ind]
sortxgen_samples = xgen_samples_1429[ind]
inc_energies = np.linspace(0, len(entarget_samples), 46).astype(int)

print(sortentarget_samples, sortentarget_samples.shape)
print(sortxtarget_samples, sortxtarget_samples.shape)
print(sortxgen_samples, sortxgen_samples.shape)

xenergies = (((sortentarget_samples[samps] / 1).flatten()).numpy().astype(int)).tolist()

minVal, maxVal = 0, 1
binwidth = (maxVal-minVal)/20

for j in range(45):
    
    # Create lists for the histogram values
    target_counts, gen_counts = [0] * 45, [0] * 45
    target_bins, gen_bins = [0] * 45, [0] * 45

    sub_target_samples = sortxtarget_samples[inc_energies[j] : inc_energies[j+1]]
    # sub_recon_samples = sortxrecon_samples[inc_energies[j] : inc_energies[j+1]]
    sub_gen_samples = sortxgen_samples[inc_energies[j] : inc_energies[j+1]]

    for i in range(45): # of layers in the cylinder - 1

        idx = (i+1)*9*16
        idxPrev = (i)*9*16
        l = idx - idxPrev
        
        target_counts[i], target_bins[i] = np.histogram((sub_target_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)
        # recon_counts[i], recon_bins[i] = np.histogram((sub_recon_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)
        gen_counts[i], gen_bins[i] = np.histogram((sub_gen_samples[:, idxPrev:idx]).sum(dim=1).numpy()/1000, bins=20)
        # mid_recon = 0.5 * (target_counts[i] + recon_counts[i])
        mid_gen = 0.5 * (target_counts[i] + gen_counts[i])

        # JSD_recon[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_recon) + KL_div(recon_counts[i].tolist(), mid_recon)))
        JSD_gen[j][i] = (0.5 * (KL_div(target_counts[i].tolist(), mid_gen) + KL_div(gen_counts[i].tolist(), mid_gen)))

        # HLD_recon[j][i] = Hellinger_dist(target_counts[i].tolist(), recon_counts[i].tolist())
        HLD_gen[j][i] = Hellinger_dist(target_counts[i].tolist(), gen_counts[i].tolist())

print(JSD_recon)
print(min(JSD_recon.flatten()))
print(JSD_recon, JSD_recon.shape)

# Plotting for JSD_gen
fig, ax = plt.subplots(figsize=(12, 6))

cax = ax.imshow(JSD_gen, interpolation='nearest', cmap='rainbow')
ax.set_aspect('equal')
ax.set_title("Ground Truth vs Model Prediction (Energy per Event)")
ax.set_yticks(np.linspace(0, 44, 10))
ax.set_yticklabels([int(x) for x in xenergies])
ax.set_ylabel("Incident Energy Ranges (MeV)")
ax.set_xlabel("Layer Number")

divider = make_axes_locatable(ax)
cax_cbar = divider.append_axes("right", size="5%", pad=0.25)
cbar = fig.colorbar(cax, cax=cax_cbar)
cbar.set_label('JSD Divergence Value')

plt.tight_layout()
plt.show()

In [ ]:
# Plotting for HLD_recon and HLD_gen
fig, ax2 = plt.subplots(figsize=(12, 6))

# Plot for HLD_recon
# cax1 = ax1.imshow(HLD_recon, interpolation='nearest', cmap='viridis')
# ax1.set_aspect('equal')
# ax1.set_yticks(np.linspace(0, 43, 10))
# ax1.set_yticklabels([int(x) for x in xenergies])
# ax1.set_ylabel("Incident Energy Ranges")
# ax1.set_xlabel("Layer Number")
# divider1 = make_axes_locatable(ax1)
# cax_cbar1 = divider1.append_axes("right", size="5%", pad=0.25)
# cbar1 = fig.colorbar(cax1, cax=cax_cbar1)
# cbar1.set_label('Hellinger Distance Value')

# Plot for HLD_gen
cax2 = ax2.imshow(HLD_gen, interpolation='nearest', cmap='viridis')
ax2.set_aspect('equal')
ax2.set_yticks(np.linspace(0, 43, 10))
ax2.set_yticklabels([int(x) for x in xenergies])
ax2.set_ylabel("Incident Energy Ranges")
ax2.set_xlabel("Layer Number")
divider2 = make_axes_locatable(ax2)
cax_cbar2 = divider2.append_axes("right", size="5%", pad=0.25)
cbar2 = fig.colorbar(cax2, cax=cax_cbar2)
cbar2.set_label('Hellinger Distance Value')

plt.tight_layout()
plt.show()


In [ ]:
HLF_1_electron.relevantLayers = [0,5,10,15,20,25,30,35,40,45]
plt.figure(figsize=(8,6))
# Create a grid of subplots
fig, axes = plt.subplots(3,3, figsize=(12, 12), sharey=False, sharex=False, tight_layout=True)
fig.text(0.5, 0.0, 'Energy per event (GeV)', ha='center', fontsize=15)
fig.text(0.0, 0.5, 'Histogram', va='center', rotation='vertical', fontsize=15)

# Iterate through the columns of X and plot histograms
for i,_ in enumerate(HLF_1_electron.relevantLayers[:-1]):
    row_index = i // 3  # Determine the row index
    col_index = i % 3   # Determine the column index
    
    ax = axes[row_index, col_index]  # Get the current subplot
    
    # Plot histogram for the current column
    idx = HLF_1_electron.relevantLayers[i+1]*9*16
    idxPrev = (HLF_1_electron.relevantLayers[i])*9*16
    l = idx - idxPrev
    ax.scatter(xtarget_samples_orig[:, idxPrev:idx].sum(dim=1).numpy()/1000, xrecon_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, marker='.', alpha=.5, color="b")
    ax.scatter(xtarget_samples_orig[:, idxPrev:idx].sum(dim=1).numpy()/1000, xgen_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000, marker='.', alpha=.5, color="orange")    
    minVal, maxVal = np.min(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000), np.max(xtarget_samples[:, idxPrev:idx].sum(dim=1).numpy()/1000) 
    ax.plot([minVal, maxVal],[minVal, maxVal], c='red', lw=2.5, label='y=x')
    if i == 0:
        # ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], title=f'{ds[part]}')
        ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], fontsize=15)
    ax.grid("True")
    
    ax.set_title(f'Layers {HLF_1_electron.relevantLayers[i]} to {HLF_1_electron.relevantLayers[i+1]-1}')

# Adjust layout and display the plots
plt.tight_layout()
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_per_layer_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
plt.scatter(xtarget_samples_orig.sum(dim=1).numpy()/1000, xrecon_samples.sum(dim=1).numpy()/1000, marker='.', alpha=.5, color='b')
plt.scatter(xtarget_samples_orig.sum(dim=1).numpy()/1000, xgen_samples.sum(dim=1).numpy()/1000, marker='.', alpha=.5, color='orange')
# plt.scatter(xtarget_samples.sum(dim=1).numpy()/1000, xgen_samples_qpu.sum(dim=1).numpy()/1000, marker=‘.’, alpha=.5, color=“cyan”)
plt.plot([0,800],[0,800], c='red', lw=2.5, label='y=x')
# plt.plot([0,20],[0,20], c=‘red’, lw=2)
# axes[0,1].set_xlabel(“GT energy per event (GeV)“)
plt.legend(['Recon', 'MC', 'QPU', 'y=x'], fontsize=17)
plt.grid('True')
plt.xlabel('GT energy per event (GeV)', fontsize=15)
plt.ylabel('Model output energy per \n event (GeV)', fontsize=15)
# plt.savefig(f’/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_scatter_{modelname}_{arch}_{datascaled}_{part}.png’)
plt.show()

In [ ]:
plt.hist((xtarget_samples[idxEnFilter,:].sum(dim=1)/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7, density=True)
plt.hist((xrecon_samples[idxEnFilter,:].sum(dim=1).numpy()/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="c", density=True)
plt.hist((xgen_samples[idxEnFilter,:].sum(dim=1).numpy()/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot", density=True)

plt.xlabel("Energy ratio", fontsize=15)
# plt.xscale('log')

plt.ylabel("Histogram", fontsize=15)
plt.legend(["GT", "Recon", "Sample", "Sample w/ QPU"])
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_ration_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

# Energy across all voxels throughout the cylinder / incidence energy

In [ ]:
print(xtarget_samples.shape, xgen_samples.shape, xgen_samples_1429.shape
     )

In [ ]:
import jetnet
# Refer to Documentation: https://jetnet.readthedocs.io/en/latest/pages/metrics.html#jetnet.evaluation.fpd
# Metrics will be implemented once dataset # and sample # matches
# jetnet.evaluation.fpd(real_features: Tensor | np.ndarray, gen_features: Tensor | np.ndarray, min_samples: int = 20000, max_samples: int = 50000, num_batches: int = 20, num_points: int = 10, normalise: bool = True, seed: int = 42)→ tuple[float, float]
# jetnet.evaluation.kpd(real_features: Tensor | np.ndarray, gen_features: Tensor | np.ndarray, num_batches: int = 10, batch_size: int = 5000, normalise: bool = True, seed: int = 42, num_threads: int | None = None)→ tuple[float, float]
fpd_met = jetnet.evaluation.fpd(xtarget_samples, xgen_samples_1429)

In [ ]:
print("FPD Metric Value: ", fpd_met)

In [ ]:
kpd_met = jetnet.evaluation.kpd(xtarget_samples, xgen_samples_1429)
print("KPD Metric Value: ", kpd_met)

In [ ]:
# plt.hist((xtarget_samples_orig.sum(dim=1)/entarget_samples_orig[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7, density=True)
# plt.hist((xrecon_samples.sum(dim=1).numpy()/entarget_samples_orig[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="c", density=True)
# plt.hist((xgen_samples.sum(dim=1).numpy()/entarget_samples_orig[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot", density=True)

plt.hist((xtarget_samples.sum(dim=1)/entarget_samples.view(-1)).numpy(), bins=30, log=True, density=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
plt.hist((xrecon_samples.sum(dim=1).numpy()/entarget_samples_orig.view(-1)).numpy(), bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="c")
plt.hist((xgen_samples.sum(dim=1).numpy()/entarget_samples_orig.view(-1)).numpy(), bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
# plt.hist((xgen_samples_1429.sum(dim=1).numpy()/entarget_samples.view(-1)).numpy(), bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")
plt.hist((xgen_samples_qpu.sum(dim=1).numpy()/entarget_samples.view(-1)).numpy(), bins=30, log=True, density=True, histtype='step', linewidth=2.5, color="purple", linestyle="dashdot")


plt.xlabel("Energy ratio", fontsize=15)
# plt.xscale('log')

plt.ylabel("Histogram", fontsize=15)
plt.legend(["GT", "Recon", "Samples", "Samples w/ QPU"])
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_ration_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

# Energy across all voxels throughout the cylinder / incidence energy

In [ ]:
print(entarget_samples_orig, entarget_samples_orig.shape, entarget_samples_orig.view(-1), entarget_samples_orig.view(-1).shape)

In [ ]:
from scipy.stats import linregress
x, y = xtarget_samples_orig.sum(dim=1), entarget_samples_orig.view(-1)
slope, intercept, r_value, p_value, std_err = linregress(x, y)
print(f"Slope: {slope}")
print(f"Intercept: {intercept}")
print(f"R-squared: {r_value**2}")
print(f"P-value: {p_value}")
print(f"Standard error: {std_err}")
plt.scatter(x, y)
plt.scatter(x, y, color='blue', label='Data points')
plt.plot(x, slope*x + intercept, color='red', label='Fitted line')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()
comb = xtarget_samples_orig.sum(dim=1)/entarget_samples_orig.view(-1)
print(entarget_samples_orig[comb > 1.5])

In [ ]:
plt.hist((xtarget_samples.sum(dim=1)/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7, density=True)
plt.hist((xrecon_samples.sum(dim=1).numpy()/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="c", density=True)
plt.hist((xgen_samples.sum(dim=1).numpy()/entarget_samples[idxEnFilter].view(-1)).numpy(), bins=30, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot", density=True)

plt.xlabel("Energy ratio", fontsize=15)
# plt.xscale('log')

plt.ylabel("Histogram", fontsize=15)
plt.legend(["GT", "Recon", "Sample", "Sample w/ QPU"])
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_ration_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

# Energy across all voxels throughout the cylinder / incidence energy

In [ ]:
plt.hist(xtarget_samples[idxEnFilter,:].sum(dim=1).numpy()/1000, bins=50, log=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7, density=True)
plt.hist(xrecon_samples[idxEnFilter,:].sum(dim=1).numpy()/1000, bins=50, log=True, histtype='step', linewidth=2.5, color="c", density=True)
plt.hist(xgen_samples[idxEnFilter,:].sum(dim=1).numpy()/1000, bins=50, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot", density=True)
# plt.hist(xgen_samples_qpu[idxEnFilter,:].sum(dim=1).numpy()/1000, bins=30, log=True, histtype='step', linewidth=2.5, color="m", linestyle="dashed", density=True)

plt.xlabel("energy per event (GeV)", fontsize=15)
# plt.xscale('log')

plt.ylabel("Histogram", fontsize=15)
plt.legend(["GT", "Recon", "Sample", "Sample w/ QPU"])
# plt.title(f'{ds[part]} \n {E_left/1000}<E_inc<{E_right/1000} (GeV)')
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_slice_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
HLF_1_electron.relevantLayers = [0,5,10,15,20,25,30,35,40,45]
plt.figure(figsize=(8,6))
# Create a grid of subplots
fig, axes = plt.subplots(3,3, figsize=(12, 12), sharey=True, sharex=False, tight_layout=True)
fig.text(0.5, 0.0, 'Energy per event (GeV)', ha='center', fontsize=15)
fig.text(0.0, 0.5, 'Histogram', va='center', rotation='vertical', fontsize=15)

# Iterate through the columns of X and plot histograms
for i,_ in enumerate(HLF_1_electron.relevantLayers[:-1]):
    row_index = i // 3  # Determine the row index
    col_index = i % 3   # Determine the column index
    
    ax = axes[row_index, col_index]  # Get the current subplot
    
    # Plot histogram for the current column
    idx = HLF_1_electron.relevantLayers[i+1]*9*16
    idxPrev = (HLF_1_electron.relevantLayers[i])*9*16
    l = idx - idxPrev
    ax.hist(xtarget_samples[idxEnFilter, idxPrev:idx].sum(dim=1).numpy()/1000,  bins=10, log=True, histtype='stepfilled', linewidth=2.5, color="b", alpha=0.7)
    ax.hist(xrecon_samples[idxEnFilter, idxPrev:idx].sum(dim=1).numpy()/1000, bins=10, log=True, histtype='step', linewidth=2.5, color="c")
    # ax.hist(xrecon_samples_2[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20, log=True, histtype='step', linewidth=2.5, color="m", linestyle="dashdot")
    ax.hist(xgen_samples[idxEnFilter, idxPrev:idx].sum(dim=1).numpy()/1000, bins=10, log=True, histtype='step', linewidth=2.5, color="orange", linestyle="dashdot")
    # ax.hist(xgen_samples_qpu[:, idxPrev:idx].sum(dim=1).numpy()/1000, bins=20, log=True, histtype='step', linewidth=2.5, color="m", linestyle="dashed")
    if i == 0:
        # ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], title=f'{ds[part]}')
        ax.legend(["GT", "Recon", "Sample", "Sample w/ QPU"], fontsize=15)
    ax.grid("True")
    
    # Set labels and title for the subplot
    # ax.set_xlabel(f'Column {i + 1}')
    # ax.set_ylabel('Frequency')
    ax.set_title(f'Layers {HLF_1_electron.relevantLayers[i]} to {HLF_1_electron.relevantLayers[i+1]-1}')

# Adjust layout and display the plots
plt.tight_layout()
# plt.savefig(f'/home/javier/Projects/CaloQVAE/figs/{modelname}/energy_per_layer_{modelname}_{arch}_{datascaled}_{part}.png')
plt.show()

In [ ]:
# GRANULARITY METRICS
def measure_single_granularity(data_tensor_1, index):
    """
    Measure the granularity of a calorimeter image given as a 1D array using PyTorch.
    
    Parameters:
    data (array-like): 1D array representing the calorimeter image.
    
    Returns:
    float: A measure of the granularity.
    """
    # Calculate the differences between consecutive elements
#     diffs_1 = (data_tensor_1[:,index:] - data_tensor_1[:,:-index])/torch.mean(data_tensor_1[:,:-index], dim = 1, keepdim=True)
    diffs_1 = (data_tensor_1[:,index:] - data_tensor_1[:,:-index])
#     std_g = torch.std(diffs_1, dim = 1)
    return diffs_1


def measure_stochastic_granularity(data_tensor):
    segment_length = 144  # Length of each segment
    num_segments = 45     # Number of segments per sample
    num_samples = data_tensor.size(0)  # Number of samples in the data tensor

    # Use PyTorch to generate a random integer array of shape (num_samples,) with values between 0 and 15
    random_array = torch.randint(0, 16, (num_samples,), dtype=torch.int64, device=data_tensor.device)

    # Multiply random_array by 9 and expand it to shape (num_samples, num_segments)
    shifts = (random_array * 9).unsqueeze(-1).expand(-1, num_segments)

    # Unfold the data tensor to create segments of length 144
    # The result is a tensor of shape (num_samples, num_segments, segment_length)
    segments = data_tensor.unfold(1, segment_length, segment_length)

    # Create an indices tensor of shape (num_samples, num_segments, segment_length)
    indices = torch.arange(segment_length, device=data_tensor.device).repeat(num_samples, num_segments, 1)
    # Adjust the indices by adding the shifts and applying modulo operation to wrap around
    indices = (indices + shifts.unsqueeze(-1)) % segment_length  # Ensure correct broadcasting

    # Gather elements from the segments tensor using the adjusted indices
    rotated_segments = torch.gather(segments, 2, indices)

    # Reshape the rotated_segments tensor back to the original shape of data_tensor
    result_tensor = rotated_segments.view(num_samples, -1)

    # Compute the difference between the original data tensor and the result tensor
    diffs = data_tensor - result_tensor

    return diffs                                    

In [ ]:
gen_data_gran = h5py.File("/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_denim-smoke-166en130.hdf5", 'r')

In [ ]:
gran_gen_samples = torch.tensor(gen_data_gran['showers'])

In [ ]:
print(xtarget_samples_orig.shape)
print(gran_gen_samples.shape)

In [ ]:
# granularity_target = measure_single_granularity(xtarget_samples,9)
# # granularity_sample = measure_single_granularity(xgen_samples,9)
# granularity_recon = measure_single_granularity(xrecon_samples,9)

granularity_target = measure_stochastic_granularity(xtarget_samples)
granularity_recon = measure_stochastic_granularity(xrecon_samples)
granularity_sample = measure_stochastic_granularity(xgen_samples)
std_target = torch.std(granularity_target, dim =1)
std_recon = torch.std(granularity_recon, dim =1)
std_sample = torch.std(granularity_sample, dim =1)

In [ ]:
combined_data = np.concatenate((std_target, std_recon))
bins = np.histogram_bin_edges(combined_data, bins=50)
plt.hist(std_target, log=True, color='red', bins=bins, linestyle='--', label='std_target', density=True, histtype='step')
plt.hist(std_recon, log=True, color='blue', bins=bins, linestyle='--', label='std_recon', density=True, histtype='step')
plt.hist(std_sample, log=True, color='purple', bins=bins, linestyle='--', label='std_sample', density=True, histtype='step')

plt.xlabel('Standard Deviation')
plt.ylabel('Density')
plt.legend()
plt.title("Granularity STD Histogram")
plt.show()

In [ ]:
index = []
error_target_list = []
error_recon_list = []
error_alt_list = []
ratio_list = []
ratio_list_alt = []
for i in range(1, 30):
    error_recon = torch.mean(torch.std(measure_single_granularity(xrecon_samples, i),dim=0))
    error_target = torch.mean(torch.std(measure_single_granularity(xtarget_samples, i),dim=0))
    error_alt = torch.mean(torch.std(measure_single_granularity(xgen_samples, i),dim=0))
    ratio = error_target/error_recon
    ratio_alt = error_target/error_alt
    error_recon_list.append(error_recon)
    error_target_list.append(error_target)
    error_alt_list.append(error_alt)
    ratio_list.append(ratio)
    ratio_list_alt.append(ratio_alt)
    index.append(i)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 8), gridspec_kw={'height_ratios': [2, 1]})

# 在第一个子图中绘制 error_gen_list 和 error_target_list
ax1.plot(index, error_target_list, label='Target', color='black')
ax1.plot(index, error_recon_list, label='Recon', color='red')
ax1.plot(index, error_alt_list, label='Samples', color='green')
ax1.legend()
ax1.set_ylabel('Mean STD')

# 在第二个子图中绘制 ratio_list
ax2.plot(index, ratio_list, label='Ratio T/G', color='red')
ax2.plot(index, ratio_list_alt, label='Ratio T/A', color='green')
ax2.legend()
ax2.set_xlabel('Shifting Index')
ax2.set_ylabel('Ratio')

plt.show()

In [ ]:
(((xtarget_samples - xrecon_samples)/(xtarget_samples+1e-5))**2).sum(dim=1).argsort()[-200:-1]

In [ ]:
print(xtarget_samples_orig, xtarget_samples_orig.shape)
print(xgen_samples, xgen_samples.shape)
print(entarget_samples_orig, entarget_samples_orig.shape)
# print(entarget_samples[entarget_samples[9500 < ]])

spec_energy = 1000000 #MeV
diff_energy = abs(entarget_samples_orig - spec_energy)
print(entarget_samples_orig[torch.nonzero(diff_energy == min(diff_energy))[0][0]])
idx = torch.nonzero(diff_energy == min(diff_energy))[0][0].item()
print(idx)
print((xtarget_samples[idx]).sum(dim=0))
print(xgen_samples[1000], xgen_samples[1000].shape)

In [ ]:
# 3D PLOTTING

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, LogNorm

# Generate the array of 6480 values (example data)
data = (xtarget_samples_orig[idx]).numpy()
print("Data sum:", data.sum())
print("Data shape:", data.shape)

# Reshape the data to 45 layers x 16 slices x 9 circles
count = 0
data_reshape = np.zeros((45, 16, 9))
print("Reshaped data shape:", data_reshape.shape)
for i in range(data_reshape.shape[0]):
    for j in range(data_reshape.shape[1]):
        for k in range(data_reshape.shape[2]):
            data_reshape[i][j][k] = data[count]
            count += 1

# Check min and max values
data_min = data_reshape.min()
data_max = data_reshape.max()
print("Data min:", data_min, "Data max:", data_max)

# Normalize the data using logarithmic normalization
norm = LogNorm(vmin=1e+0, vmax=1e+4)  # Adding a small value to avoid log(0)

print("Normalized data shape:", data_reshape.shape)

# Create the cylindrical coordinates
theta = np.linspace(0, 2 * np.pi, 17)
r = np.linspace(0, 1, 10)
theta, r = np.meshgrid(theta, r)

# Create the X, Y, Z coordinates for the cylinder rotated on its side
Y = r * np.cos(theta) * 40
Z = r * np.sin(theta) * 40

# Create a figure and 3D axis
fig = plt.figure(figsize=(30, 20))
ax = fig.add_subplot(111, projection='3d')

# Plot each layer separately

layer_start = 40
layer_end = 45
for i in range(layer_start, layer_end):
    X = np.full_like(Y, i)  # Each layer is at a constant X value
    values = data_reshape[i, :, :].T  # Transpose to match the shape
    values += 1e-16 # prevent log 0
    
    # Normalize and apply colormap
    norm_values = norm(values)+1e-16
    print(f"Layer {i}, normalized values min: {norm_values.min()}, max: {norm_values.max()}")
    facecolors = plt.cm.rainbow(norm_values)  # Apply colormap to normalized values
    
    facecolors[..., -1] = 1 # Set the alpha channel to 0.3 for transparency
    
    surf = ax.plot_surface(X, Y, Z, facecolors=facecolors, rstride=1, cstride=1, shade=False, edgecolor='k')
    # surf = ax.plot_surface(X, Y, Z, facecolors=facecolors+1e-16, rstride=1, cstride=1, shade=False)

# Add a color bar
mappable = plt.cm.ScalarMappable(cmap=plt.cm.rainbow, norm=norm)
mappable.set_array(data)
cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, pad=-0.05)
cbar.set_label('Energy (MeV)')

# Add labels
ax.set_xlabel('Layers')
ax.set_ylabel('x (mm)')
ax.set_zlabel('y (mm)', rotation=90)
ax.set_xlim(44, 0)
# ax.set_ylim(-40, 40)
# ax.set_zlim(-40, 40)

ax.set_box_aspect([44, 44, 36])

# plt.tight_layout()

# Set the view angle
ax.view_init(elev=0, azim=70)  # Adjust these values to change the view
ax.grid(False)
ax.axis('off')


plt.title('EM Shower in Calorimeter')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, LogNorm

# Generate the array of 6480 values (example data)
data = (xtarget_samples_orig[idx]).numpy()
print("Data sum:", data.sum())
print("Data shape:", data.shape)

# Reshape the data to 45 layers x 16 slices x 9 circles
count = 0
data_reshape = np.zeros((45, 16, 9))
print("Reshaped data shape:", data_reshape.shape)
for i in range(data_reshape.shape[0]):
    for j in range(data_reshape.shape[1]):
        for k in range(data_reshape.shape[2]):
            data_reshape[i][j][k] = data[count]
            count += 1

# Check min and max values
data_min = data_reshape.min()
data_max = data_reshape.max()
print("Data min:", data_min, "Data max:", data_max)

# Normalize the data using logarithmic normalization
norm = LogNorm(vmin=1e+0, vmax=1e+4)  # Adding a small value to avoid log(0)

print("Normalized data shape:", data_reshape.shape)

# Create the rectangular coordinates
x = np.linspace(-1, 1, 17)
y = np.linspace(-1, 1, 10)
x, y = np.meshgrid(x, y)

# Create the X, Y, Z coordinates for the rectangular prism
Y = y * 40
Z = x * 40

# Create a figure and 3D axis
fig = plt.figure(figsize=(32, 32))
ax = fig.add_subplot(111, projection='3d')

# Plot each layer separately
layer_start = 10
for i in range(layer_start, 45):
    X = np.full_like(Y, i)  # Each layer is at a constant X value
    values = data_reshape[i, :, :].T  # Transpose to match the shape
    values += 1e-16  # Prevent log 0
    
    # Normalize and apply colormap
    norm_values = norm(values)
    print(f"Layer {i}, normalized values min: {norm_values.min()}, max: {norm_values.max()}")
    facecolors = plt.cm.rainbow(norm_values)  # Apply colormap to normalized values
    
    facecolors[..., -1] = 1  # Set the alpha channel to 0.9 for transparency
    
    surf = ax.plot_surface(X, Y, Z, facecolors=facecolors, rstride=1, cstride=1, shade=False, edgecolor='k')

# Add a color bar
mappable = plt.cm.ScalarMappable(cmap=plt.cm.rainbow, norm=norm)
mappable.set_array(data)
cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, pad=0.05)
cbar.set_label('Energy (MeV)')

# Add labels
ax.set_xlabel('Layers')
ax.set_ylabel('x (mm)')
ax.set_zlabel('y (mm)', rotation=90)
ax.set_xlim(44, 0)
# ax.set_ylim(-40, 40)
# ax.set_zlim(-40, 40)

ax.set_box_aspect([45, 9, 16])

# plt.tight_layout()

# Set the view angle
ax.view_init(elev=-5, azim=4, roll = 89.2
            )  # Adjust these values to change the view
ax.grid(False)
ax.axis('off')

plt.title('EM Shower in Calorimeter')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import Normalize, LogNorm

# Generate the array of 6480 values (example data)
data = (xtarget_samples_orig[idx]).numpy()

# Reshape the data to 45 layers x 16 slices x 9 circles
count = 0
data_reshape = np.zeros((45, 16, 9))
print("Reshaped data shape:", data_reshape.shape)
for i in range(data_reshape.shape[0]):
    for j in range(data_reshape.shape[1]):
        for k in range(data_reshape.shape[2]):
            data_reshape[i][j][k] = data[count]
            count += 1

data = data_reshape.copy()
print(data.shape)
mid = data.shape[-2] // 2
shift = np.concatenate((data[:,mid:, [0]], data[:,:mid, [0]]), axis=-2)
print(shift.shape)
data = np.concatenate((shift, data), axis=-1)
print(data.shape)
data = np.pad(data, ((0, 0), (1,1), (0, 0)), mode='wrap')
print("Data sum:", data.sum())
print("Data shape:", data.shape)
data_reshape = data

# Check min and max values
data_min = data_reshape.min()
data_max = data_reshape.max()
print("Data min:", data_min, "Data max:", data_max)

# Normalize the data using logarithmic normalization
norm = LogNorm(vmin=1e+0, vmax=1e+4)  # Adding a small value to avoid log(0)

print("Normalized data shape:", data_reshape.shape)

# Create the rectangular coordinates
x = np.linspace(-1, 1, 19)
y = np.linspace(-1, 1, 11)
x, y = np.meshgrid(x, y)

# Create the X, Y, Z coordinates for the rectangular prism
Y = y * 40
Z = x * 40

# Create a figure and 3D axis
fig = plt.figure(figsize=(32, 32))
ax = fig.add_subplot(111, projection='3d')

# Plot each layer separately
layer_start = 10
for i in range(layer_start, 45):
    X = np.full_like(Y, i)  # Each layer is at a constant X value
    values = data_reshape[i, :, :].T  # Transpose to match the shape
    values += 1e-16  # Prevent log 0
    
    # Normalize and apply colormap
    norm_values = norm(values)
    print(f"Layer {i}, normalized values min: {norm_values.min()}, max: {norm_values.max()}")
    facecolors = plt.cm.rainbow(norm_values)  # Apply colormap to normalized values
    
    facecolors[..., -1] = 1  # Set the alpha channel to 0.9 for transparency
    
    surf = ax.plot_surface(X, Y, Z, facecolors=facecolors, rstride=1, cstride=1, shade=False, edgecolor='k')

# Add a color bar
mappable = plt.cm.ScalarMappable(cmap=plt.cm.rainbow, norm=norm)
mappable.set_array(data.reshape(-1))
cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, pad=0.05)
cbar.set_label('Energy (MeV)')

# Add labels
ax.set_xlabel('Layers')
ax.set_ylabel('x (mm)')
ax.set_zlabel('y (mm)', rotation=90)
ax.set_xlim(44, 0)
# ax.set_ylim(-40, 40)
# ax.set_zlim(-40, 40)

ax.set_box_aspect([45, 30, 54])

# plt.tight_layout()

# Set the view angle
ax.view_init(elev=-0, azim=0, roll = 90
            )  # Adjust these values to change the view
ax.grid(False)
ax.axis('off')

plt.title('EM Shower in Calorimeter')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Generate the array of 6480 values (example data)
data = np.random.random(6480) * 100  # Example data
print("Data sum:", data.sum())
print("Data shape:", data.shape)

# Reshape the data to 45 layers x 16 slices x 9 circles
data_reshape = data.reshape((45, 16, 9))
print("Reshaped data shape:", data_reshape.shape)

# Check min and max values
data_min = data_reshape.min()
data_max = data_reshape.max()
print("Data min:", data_min, "Data max:", data_max)

# Normalize the data using logarithmic normalization
norm = LogNorm(vmin=1e-1, vmax=data_max)

# Flatten the data for normalization
flattened_data = data_reshape.flatten()

# Normalize data for color mapping
norm_data = norm(flattened_data)

# Reshape normalized data to match the original shape
norm_data_reshaped = norm_data.reshape(data_reshape.shape)

# Create cylindrical coordinates
theta = np.linspace(0, 2 * np.pi, 16, endpoint=False)
r = np.linspace(0, 1, 9, endpoint=False)
theta, r = np.meshgrid(theta, r)

# Convert cylindrical to Cartesian coordinates
x = r * np.cos(theta)
y = r * np.sin(theta)
z = np.arange(45)

# Repeat coordinates for each layer
x = np.repeat(x[np.newaxis, :, :], 45, axis=0)
y = np.repeat(y[np.newaxis, :, :], 45, axis=0)
z = z[:, np.newaxis, np.newaxis] * np.ones((1, 16, 9))

# Create a figure and 3D axis
fig = plt.figure(figsize=(30, 20))
ax = fig.add_subplot(111, projection='3d')

# Create a boolean array for where the data is non-zero
filled = data_reshape > 0

# Create the voxel plot
colors = plt.cm.viridis(norm_data_reshaped)
colors[..., -1] = 0.5  # Set alpha (transparency)

# Plot the voxels
ax.voxels(filled, facecolors=colors, edgecolor='k')

# Add a color bar
mappable = plt.cm.ScalarMappable(norm=norm, cmap=plt.cm.viridis)
mappable.set_array(flattened_data)
cbar = fig.colorbar(mappable, ax=ax, shrink=0.5, aspect=10, pad=-0.05)
cbar.set_label('Energy (MeV)')

# Add labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.set_box_aspect([1, 1, 1])

# Set the view angle
ax.view_init(elev=30, azim=30)
ax.grid(False)
ax.axis('off')

plt.title('EM Shower in Calorimeter')
plt.show()


In [ ]:
import os

dir = f'/home/luian1/CaloQVAE/figs/{modelname}'
HLF_1_electron.relevantLayers = [0,5,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,30,35,40,44]

if not os.path.exists(dir):
    os.makedirs(dir)

HLF_1_electron.DrawSingleShower(xtarget_samples[idx,:].detach().cpu().numpy(), filename=f'/home/luian1/CaloQVAE/figs/{modelname}/target_a_{idx}_{modelname}_{arch}_{datascaled}_{part}.png')
HLF_1_electron.DrawSingleShower(xtarget_samples[idx,:].detach().cpu().numpy(), filename=None, vmax=1e+4, vmin=1e+0, cmap='rainbow')

In [ ]:
# HLF_1_electron.DrawSingleShower(xrecon_samples[idx,:].detach().cpu().numpy(), filename=f'/home/luian1/Projects/CaloQVAE/figs/{modelname}/target_a_{idx}_{modelname}_{arch}_{datascaled}_{part}.png')
HLF_1_electron.DrawSingleShower(xrecon_samples[idx,:].detach().cpu().numpy(), filename=None, vmax=1e+4, vmin=1e+0, cmap='rainbow')

In [ ]:
HLF_1_electron.DrawSingleShower(xgen_samples[idx,:].detach().cpu().numpy(), filename=f'/home/luian1/CaloQVAE/figs/{modelname}/target_a_{idx}_{modelname}_{arch}_{datascaled}_{part}.png')
HLF_1_electron.DrawSingleShower(xgen_samples[idx,:].detach().cpu().numpy(), filename=None, vmax=1e+4, vmin=1e+0, cmap='rainbow')

In [ ]:
# Use test dataset /fast_scratch/QVAE/test_data/dataset_2_2.hdf5
# array from hdf5 -> convert to torch tensor

In [ ]:
xtarget_samples = []
xrecon_samples = []
xgen_samples = []
scaled = True
entarget_samples = []

def itr_merge(*itrs):
    for itr in itrs:
        for v in itr:
            yield v
            
# for xx in train_loader:
for xx in new_test_loader:
    in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
    
#     fwd_output = engine.model((in_data, true_energy), False)
    
#     if scaled:
#         in_data = torch.tensor(engine._data_mgr.inv_transform(in_data.detach().cpu().numpy()))
        
        
#         recon_data = torch.tensor(engine._data_mgr.inv_transform(fwd_output.output_activations.detach().cpu().numpy()))
        
        ## This is how youi generate data using model
        # ================================
        engine._model.sampler._batch_size = true_energy.shape[0]
        sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy) #engine._model.generate_samples_qpu(num_samples=128, true_energy=true_energy[:128])
        engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
        sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
        # ================================
    
    else:
        raise Exception("Script only supports scaled dataset at the moment...")


    xtarget_samples.append(in_data.detach().cpu())
    xrecon_samples.append( recon_data.detach().cpu())
    xgen_samples.append( sample_data.detach().cpu())
    entarget_samples.append(true_energy.detach().cpu())
    
    
xtarget_samples = torch.cat(xtarget_samples, dim=0)
xrecon_samples = torch.cat(xrecon_samples, dim=0)
xgen_samples = torch.cat(xgen_samples, dim=0)
entarget_samples = torch.cat(entarget_samples, dim=0)

In [ ]:
samp = next(iter(val_loader))

In [ ]:
samp[0][0].shape

In [ ]:
###########Create Synth data
print(modelname)

In [ ]:
print(train_loader)
print(len(train_loader))

In [ ]:
engen_samples2 = []
xgen_samples2 = []
with torch.no_grad():
    for i,xx in enumerate(train_loader):
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        if reducedata:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
        elif scaled:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
        else:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = sample_data.detach().cpu()*1000

        xgen_samples2.append(sample_data.detach().cpu())
        engen_samples2.append(true_energy.detach().cpu())
        
    for i,xx in enumerate(val_loader):
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        if reducedata:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
        elif scaled:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
        else:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = sample_data.detach().cpu()*1000

        xgen_samples2.append( sample_data.detach().cpu())
        engen_samples2.append(true_energy.detach().cpu())

    for i,xx in enumerate(test_loader):
        in_data, true_energy, in_data_flat = engine._preprocess(xx[0],xx[1])
        if reducedata:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples_cond(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = engine._reduceinv(sample_data, sample_energies, R=R)
        elif scaled:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy, measure_time=True)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = torch.tensor(engine._data_mgr.inv_transform(sample_data.detach().cpu().numpy()))
        else:
            engine._model.sampler._batch_size = true_energy.shape[0]
            sample_energies, sample_data = engine._model.generate_samples(num_samples=true_energy.shape[0], true_energy=true_energy)
            engine._model.sampler._batch_size = engine._config.engine.rbm_batch_size
            sample_data = sample_data.detach().cpu()*1000

        xgen_samples2.append( sample_data.detach().cpu())
        engen_samples2.append(true_energy.detach().cpu())
        
        # if i > 30:
        #     break

xgen_samples2 = torch.cat(xgen_samples2, dim=0)
engen_samples2 = torch.cat(engen_samples2, dim=0)

In [ ]:
# import torch
import h5py

# Convert tensors to numpy arrays as h5py does not support PyTorch tensors directly
tensor1_np = xgen_samples2[:100000,:].numpy()
tensor2_np = engen_samples2[:100000,:].numpy()

# Create a new HDF5 file
with h5py.File(f'/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_{modelname}en130.hdf5', 'w') as f:
    # Create datasets for your tensors
    f.create_dataset('showers', data=tensor1_np)
    f.create_dataset('incidence energy', data=tensor2_np)

In [ ]:
import jetnet
# Refer to Documentation: https://jetnet.readthedocs.io/en/latest/pages/metrics.html#jetnet.evaluation.fpd
# Metrics will be implemented once dataset # and sample # matches
# jetnet.evaluation.fpd(real_features: Tensor | np.ndarray, gen_features: Tensor | np.ndarray, min_samples: int = 20000, max_samples: int = 50000, num_batches: int = 20, num_points: int = 10, normalise: bool = True, seed: int = 42)→ tuple[float, float]
# jetnet.evaluation.kpd(real_features: Tensor | np.ndarray, gen_features: Tensor | np.ndarray, num_batches: int = 10, batch_size: int = 5000, normalise: bool = True, seed: int = 42, num_threads: int | None = None)→ tuple[float, float]

def get_HLData(voxels, e_inc):
    

fpd_met = jetnet.evaluation.fpd(xtarget_samples, xgen_samples_1429)